In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R

In [ ]:
# Generate a synthetic trajectory (Replace this with actual robot trajectory data)
num_points = 50
t = np.linspace(0, 2 * np.pi, num_points)

# Example trajectory: A helical path with orientation changes
x = np.cos(t)
y = np.sin(t)
z = t / (2 * np.pi)
roll = np.sin(t) * np.pi / 4
pitch = np.cos(t) * np.pi / 4
yaw = t

# Convert roll, pitch, yaw to rotation matrices
poses = np.column_stack((x, y, z, roll, pitch, yaw))

In [ ]:
# Plot the trajectory
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(poses[:, 0], poses[:, 1], poses[:, 2], 'b-', label="Trajectory")

# Plot orientation frames at selected points
for i in range(0, num_points, 5):  # Plot every 5th frame for clarity
    pos = poses[i, :3]
    rpy = poses[i, 3:]
    rot_matrix = R.from_euler('xyz', rpy).as_matrix()

    # Define frame axes
    scale = 0.1
    for j, color in enumerate(['r', 'g', 'b']):  # X=Red, Y=Green, Z=Blue
        axis = rot_matrix[:, j] * scale
        ax.quiver(pos[0], pos[1], pos[2], axis[0], axis[1], axis[2], color=color)

# Labels and title
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("3D Trajectory with Orientation Frames")
ax.legend()
plt.show()


In [ ]:
import plotly.graph_objects as go

# Create a 3D scatter plot for the trajectory
fig = go.Figure()

# Add the trajectory line
fig.add_trace(go.Scatter3d(
    x=poses[:, 0], y=poses[:, 1], z=poses[:, 2],
    mode='lines',
    line=dict(color='blue', width=3),
    name="Trajectory"
))

# Add orientation frames at selected points
for i in range(0, num_points, 5):  # Every 5th frame for clarity
    pos = poses[i, :3]
    rpy = poses[i, 3:]
    rot_matrix = R.from_euler('xyz', rpy).as_matrix()
    scale = 0.1

    # Define and add frame axes (X=Red, Y=Green, Z=Blue)
    for j, color in enumerate(['red', 'green', 'blue']):
        axis = rot_matrix[:, j] * scale
        fig.add_trace(go.Scatter3d(
            x=[pos[0], pos[0] + axis[0]],
            y=[pos[1], pos[1] + axis[1]],
            z=[pos[2], pos[2] + axis[2]],
            mode='lines',
            line=dict(color=color, width=4),
            showlegend=False
        ))

# Set figure layout
fig.update_layout(
    title="Interactive 3D Trajectory with Orientation Frames",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show interactive plot
fig.show()


In [ ]:
import h5py
import numpy as np
import pyvista as pv
from tqdm import tqdm  # Import tqdm for progress bars
import os

# Explicitly set PyVista backend to standalone
pv.global_theme.jupyter_backend = 'static'

# Path to the dataset
file_path = "demo_duck_feb12.hdf5"
# f_org = h5py.File(dataset_path, "r")

# # Get the list of demos
# f_org

In [ ]:
with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())

    for demo_name in demo_keys:
        demo_data = f["data"][demo_name]

        actions = demo_data["actions"][:]  # Action data
        
        # Extract different observation types
        obs_group = demo_data["obs"]
        if "eye_in_hand_rgb" in obs_group:
            images = obs_group["eye_in_hand_rgb"][:]  # Camera images
            print(f"{demo_name} -> Image shape: {images.shape}")
        
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # Robot proprioceptive state
            print(f"{demo_name} -> Robot State shape: {robot_states.shape}")

        # Print first action and state
        # print(f"{demo_name} -> First Action: {actions[0]}")
        if "robot_state" in obs_group:
            print(f"{demo_name} -> First Robot State: {robot_states[0]}")


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Build a cube mesh in Plotly with Rotation
###############################################################################

def make_cube_mesh(cx, cy, cz, rotation, side=0.01, color="blue", opacity=0.8):
    """
    Return a rotated Plotly Mesh3d cube of edge length 'side'
    centered at (cx, cy, cz) with given rotation.
    """
    half = side / 2.0

    # Define the 8 corners of a unit cube
    corners = np.array([
        [-half, -half, -half], [ -half, -half, half], 
        [-half,  half, -half], [-half,  half, half], 
        [ half, -half, -half], [ half, -half, half], 
        [ half,  half, -half], [ half,  half, half]
    ])

    # Rotate the cube using the provided rotation matrix
    rotated_corners = rotation.apply(corners)

    # Shift to the new position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Triangular faces (two per face)
    I = [0, 0, 1, 1, 2, 2, 4, 4, 0, 0, 2, 2]
    J = [2, 6, 5, 7, 3, 7, 6, 7, 4, 5, 3, 7]
    K = [6, 4, 7, 3, 7, 2, 7, 5, 5, 1, 7, 6]

    return go.Mesh3d(
        x=rotated_corners[:, 0],
        y=rotated_corners[:, 1],
        z=rotated_corners[:, 2],
        i=I, j=J, k=K,
        color=color,
        opacity=opacity,
        flatshading=True,
        showlegend=False
    )

def random_color(alpha=0.8):
    """ Generate a random RGBA color for Plotly """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgba({r},{g},{b},{alpha})"

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:2]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Create interactive Plotly figure with RPY variations
###############################################################################

fig = go.Figure()

deg_to_rad = np.pi / 180.0
max_deg = 5.0        # Maximum angular variation in degrees
max_side = 0.01      # Maximum cube side length

# Max RPY offsets
max_roll, max_pitch, max_yaw = 10 * deg_to_rad, 5 * deg_to_rad, 5 * deg_to_rad

# Exponential decay rate
lambda_decay = 3.0  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay function for faster reduction
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # Shrinking angle offsets
        delta_angle = fraction * max_deg * deg_to_rad
        roll_offset = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset = fraction * max_yaw

        # Shrinking cube size
        side_len = fraction * max_side

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Generate new orientations (Roll, Pitch, Yaw)
        for roll_adj in [-roll_offset, 0.0, roll_offset]:
            for pitch_adj in [-pitch_offset, 0.0, pitch_offset]:
                for yaw_adj in [-yaw_offset, 0.0, yaw_offset]:
                    rotation_matrix = R.from_euler('xyz', [roll + roll_adj, pitch + pitch_adj, yaw + yaw_adj])

                    # Azimuth and Elevation variations
                    for az_offset in [-delta_angle, 0.0, delta_angle]:
                        for el_offset in [-delta_angle, 0.0, delta_angle]:
                            az_new = az + az_offset
                            el_new = el + el_offset
                            x_new, y_new, z_new = sph2cart(r, az_new, el_new)

                            if side_len > 1e-9:
                                fig.add_trace(make_cube_mesh(
                                    cx=x_new, cy=y_new, cz=z_new,
                                    rotation=rotation_matrix,
                                    side=side_len,
                                    color=random_color(),
                                    opacity=0.8
                                ))

# Final layout adjustments
fig.update_layout(
    title="Full 6D Pose Variation with RPY (Shrinking to Target)",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(
            projection=dict(type="orthographic")
        )
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# fig.show()
# Save interactive HTML
fig.write_html("trajectory_visualization.html")

# Save static image (PNG)
fig.write_image("trajectory_visualization.png", width=1200, height=800)


In [ ]:
import random
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Function to Create a Rotated Cube
###############################################################################

def create_rotated_cube(cx, cy, cz, rotation, side=0.01):
    """ Generate 3D cube vertices with rotation """
    half = side / 2.0

    # Define the 8 corners of a unit cube
    corners = np.array([
        [-half, -half, -half], [ -half, -half, half], 
        [-half,  half, -half], [-half,  half, half], 
        [ half, -half, -half], [ half, -half, half], 
        [ half,  half, -half], [ half,  half, half]
    ])

    # Rotate the cube using the given rotation matrix
    rotated_corners = rotation.apply(corners)

    # Shift to the new position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define the 6 cube faces using rotated corners
    faces = [
        [rotated_corners[j] for j in [0, 1, 3, 2]],  # Left
        [rotated_corners[j] for j in [4, 5, 7, 6]],  # Right
        [rotated_corners[j] for j in [0, 2, 6, 4]],  # Bottom
        [rotated_corners[j] for j in [1, 3, 7, 5]],  # Top
        [rotated_corners[j] for j in [0, 1, 5, 4]],  # Front
        [rotated_corners[j] for j in [2, 3, 7, 6]]   # Back
    ]

    return faces

def random_color(alpha=0.7):
    """ Generate a random color with transparency """
    r, g, b = np.random.rand(3)
    return (r, g, b, alpha)

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:1]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Generate 3D Plot (Static, No Zooming/Panning)
###############################################################################

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

deg_to_rad = np.pi / 180.0
max_deg = 5.0        # Maximum angular variation in degrees
max_side = 0.01      # Maximum cube side length

# Max RPY offsets
max_roll, max_pitch, max_yaw = 10 * deg_to_rad, 5 * deg_to_rad, 5 * deg_to_rad

# Exponential decay rate
lambda_decay = 3.0  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay function for faster reduction
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # Shrinking angle offsets
        delta_angle = fraction * max_deg * deg_to_rad
        roll_offset = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset = fraction * max_yaw

        # Shrinking cube size
        side_len = fraction * max_side

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Generate new orientations (Roll, Pitch, Yaw)
        for roll_adj in [-roll_offset, 0.0, roll_offset]:
            for pitch_adj in [-pitch_offset, 0.0, pitch_offset]:
                for yaw_adj in [-yaw_offset, 0.0, yaw_offset]:
                    rotation_matrix = R.from_euler('xyz', [roll + roll_adj, pitch + pitch_adj, yaw + yaw_adj])

                    # Azimuth and Elevation variations
                    for az_offset in [-delta_angle, 0.0, delta_angle]:
                        for el_offset in [-delta_angle, 0.0, delta_angle]:
                            az_new = az + az_offset
                            el_new = el + el_offset
                            x_new, y_new, z_new = sph2cart(r, az_new, el_new)

                            if side_len > 1e-9:
                                cube_faces = create_rotated_cube(x_new, y_new, z_new, rotation_matrix, side_len)
                                ax.add_collection3d(Poly3DCollection(cube_faces, facecolors=random_color(), alpha=0.7))

# Set axis labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

# Set equal aspect ratio
ax.set_box_aspect([1, 1, 1])

# Remove interactive zooming and panning
ax.set_proj_type('ortho')

# Save the static figure
plt.savefig("trajectory_visualization.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
import random
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Generate a Rotated Cube
###############################################################################

def create_rotated_cube(cx, cy, cz, rotation, side=0.005):
    """ Generate 3D cube vertices with rotation """
    half = side / 2.0

    # Define the 8 corners of a unit cube
    corners = np.array([
        [-half, -half, -half], [-half, -half, half],
        [-half,  half, -half], [-half,  half, half],
        [ half, -half, -half], [ half, -half, half],
        [ half,  half, -half], [ half,  half, half]
    ])

    # Rotate the cube using the provided rotation matrix
    rotated_corners = rotation.apply(corners)

    # Shift to the new position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define the 6 cube faces using rotated corners
    faces = [
        [rotated_corners[j] for j in [0, 1, 3, 2]],  # Left
        [rotated_corners[j] for j in [4, 5, 7, 6]],  # Right
        [rotated_corners[j] for j in [0, 2, 6, 4]],  # Bottom
        [rotated_corners[j] for j in [1, 3, 7, 5]],  # Top
        [rotated_corners[j] for j in [0, 1, 5, 4]],  # Front
        [rotated_corners[j] for j in [2, 3, 7, 6]]   # Back
    ]

    return faces

def random_color(alpha=0.7):
    """ Generate a random color with transparency """
    return (np.random.rand(), np.random.rand(), np.random.rand(), alpha)

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:2]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Optimized 3D Plot (Static, No Zooming/Panning)
###############################################################################

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

deg_to_rad = np.pi / 180.0
max_deg = 3.0        # Maximum angular variation in degrees
max_side = 0.05     # Maximum cube side length

# Max RPY offsets
max_roll, max_pitch, max_yaw = 5 * deg_to_rad, 3 * deg_to_rad, 3 * deg_to_rad

# Exponential decay rate
lambda_decay = 1.2  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay function for faster reduction
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # Shrinking angle offsets
        delta_angle = fraction * max_deg * deg_to_rad
        roll_offset = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset = fraction * max_yaw

        # Shrinking cube size
        side_len = fraction * max_side

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Generate one negative and one positive variation per axis
        for az_offset in [-delta_angle, delta_angle]:
            for el_offset in [-delta_angle, delta_angle]:
                az_new = az + az_offset
                el_new = el + el_offset
                x_new, y_new, z_new = sph2cart(r, az_new, el_new)

                # Rotate cube based on varying RPY
                for roll_adj, pitch_adj, yaw_adj in [
                    (-roll_offset, -pitch_offset, -yaw_offset),
                    ( roll_offset,  pitch_offset,  yaw_offset)
                ]:
                    rotation_matrix = R.from_euler('xyz', [roll + roll_adj, pitch + pitch_adj, yaw + yaw_adj])

                    if side_len > 1e-9:
                        cube_faces = create_rotated_cube(x_new, y_new, z_new, rotation_matrix, side_len)
                        ax.add_collection3d(Poly3DCollection(cube_faces, facecolors=random_color(), alpha=0.7))

# Set axis labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

# Set equal aspect ratio
ax.set_box_aspect([1, 1, 1])

# Remove interactive zooming and panning
ax.set_proj_type('ortho')

# Save the static figure
plt.savefig("trajectory_visualization.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
import random
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Generate a Rotated Cube
###############################################################################

def create_rotated_cube(cx, cy, cz, rotation, side=0.005):
    """ Generate 3D cube vertices with rotation """
    half = side / 2.0

    # Define the 8 corners of a unit cube
    corners = np.array([
        [-half, -half, -half], [-half, -half, half],
        [-half,  half, -half], [-half,  half, half],
        [ half, -half, -half], [ half, -half, half],
        [ half,  half, -half], [ half,  half, half]
    ])

    # Rotate the cube using the provided rotation matrix
    rotated_corners = rotation.apply(corners)

    # Shift to the new position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define the 6 cube faces using rotated corners
    faces = [
        [rotated_corners[j] for j in [0, 1, 3, 2]],  # Left
        [rotated_corners[j] for j in [4, 5, 7, 6]],  # Right
        [rotated_corners[j] for j in [0, 2, 6, 4]],  # Bottom
        [rotated_corners[j] for j in [1, 3, 7, 5]],  # Top
        [rotated_corners[j] for j in [0, 1, 5, 4]],  # Front
        [rotated_corners[j] for j in [2, 3, 7, 6]]   # Back
    ]

    return faces

def random_color(alpha=0.7):
    """ Generate a random color with transparency """
    return (np.random.rand(), np.random.rand(), np.random.rand(), alpha)

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:2]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Generate 3D Plot with Blind Variations in Viewpoints
###############################################################################

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

deg_to_rad = np.pi / 180.0
max_deg = 3.0        # Maximum angular variation in degrees
max_side = 0.07     # Maximum cube side length

# Max RPY offsets
max_roll, max_pitch, max_yaw = 10 * deg_to_rad, 5 * deg_to_rad, 5 * deg_to_rad

# Exponential decay rate
lambda_decay = 1.2  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay function for faster reduction
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # Shrinking angle offsets
        delta_angle = fraction * max_deg * deg_to_rad
        roll_offset = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset = fraction * max_yaw

        # Shrinking cube size
        side_len = fraction * max_side

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Apply small random variations in azimuth and elevation
        az_variation = az + (random.choice([-1, 1]) * delta_angle)
        el_variation = el + (random.choice([-1, 1]) * delta_angle)
        x_new, y_new, z_new = sph2cart(r, az_variation, el_variation)

        # Apply small random variations in roll, pitch, yaw
        roll_var = roll + (random.choice([-1, 1]) * roll_offset)
        pitch_var = pitch + (random.choice([-1, 1]) * pitch_offset)
        yaw_var = yaw + (random.choice([-1, 1]) * yaw_offset)

        rotation_matrix = R.from_euler('xyz', [roll_var, pitch_var, yaw_var])

        if side_len > 1e-9:
            cube_faces = create_rotated_cube(x_new, y_new, z_new, rotation_matrix, side_len)
            ax.add_collection3d(Poly3DCollection(cube_faces, facecolors=random_color(), alpha=0.7))

# Set axis labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

# Set equal aspect ratio
ax.set_box_aspect([1, 1, 1])

# Remove interactive zooming and panning
ax.set_proj_type('ortho')

# Save the static figure
plt.savefig("viewpoint_visualization.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Function to Create a Cube Mesh in Plotly
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005, color="blue"):
    """ Create a 3D cube in Plotly centered at (cx, cy, cz) with rotation """

    half = side / 2.0

    # Define cube corners
    corners = np.array([
        [-half, -half, -half], [-half, -half, half], 
        [-half,  half, -half], [-half,  half, half], 
        [ half, -half, -half], [ half, -half, half], 
        [ half,  half, -half], [ half,  half, half]
    ])

    # Apply rotation
    rotated_corners = rotation.apply(corners)

    # Shift to center position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define cube faces (triangular mesh)
    faces = [
        [0, 1, 3, 2], [4, 5, 7, 6],  # Front & Back
        [0, 2, 6, 4], [1, 3, 7, 5],  # Left & Right
        [0, 1, 5, 4], [2, 3, 7, 6]   # Bottom & Top
    ]

    return go.Mesh3d(
        x=rotated_corners[:, 0],
        y=rotated_corners[:, 1],
        z=rotated_corners[:, 2],
        i=[face[0] for face in faces],
        j=[face[1] for face in faces],
        k=[face[2] for face in faces],
        color=color,
        opacity=0.8,
        flatshading=True,
        showlegend=False
    )

def random_color():
    """ Generate a random color in Plotly format """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgb({r},{g},{b})"

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:2]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Generate Interactive 3D Plot in Plotly
###############################################################################

fig = go.Figure()

deg_to_rad = np.pi / 180.0
max_deg = 5.0        # Maximum angular variation in degrees
max_side = 0.05     # Maximum cube side length

# Max RPY offsets
max_roll, max_pitch, max_yaw = 10 * deg_to_rad, 5 * deg_to_rad, 5 * deg_to_rad

# Exponential decay rate
lambda_decay = 1.5  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay function for shrinking variation
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # Shrinking angle offsets
        delta_angle = fraction * max_deg * deg_to_rad
        roll_offset = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset = fraction * max_yaw

        # Shrinking cube size
        side_len = fraction * max_side

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Apply small random variations in azimuth and elevation
        az_variation = az + (random.choice([-1, 1]) * delta_angle)
        el_variation = el + (random.choice([-1, 1]) * delta_angle)
        x_new, y_new, z_new = sph2cart(r, az_variation, el_variation)

        # Apply small random variations in roll, pitch, yaw
        roll_var = roll + (random.choice([-1, 1]) * roll_offset)
        pitch_var = pitch + (random.choice([-1, 1]) * pitch_offset)
        yaw_var = yaw + (random.choice([-1, 1]) * yaw_offset)

        rotation_matrix = R.from_euler('xyz', [roll_var, pitch_var, yaw_var])

        if side_len > 1e-9:
            fig.add_trace(make_cube(x_new, y_new, z_new, rotation_matrix, side_len, random_color()))

# Set figure layout
fig.update_layout(
    title="Interactive Viewpoint Generation for Goal Object Visibility",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(projection=dict(type="perspective"))  # Ensures proper 3D depth
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Save interactive figure as HTML
fig.write_html("interactive_viewpoints.html")

# Show interactive plot
fig.show()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Function to Create a Cube with Edges in Plotly
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005, color="blue"):
    """ Create a visible 3D cube using Scatter3D edges """
    half = side / 2.0

    # Define cube corners
    corners = np.array([
        [-half, -half, -half], [-half, -half, half],
        [-half,  half, -half], [-half,  half, half],
        [ half, -half, -half], [ half, -half, half],
        [ half,  half, -half], [ half,  half, half]
    ])

    # Apply rotation
    rotated_corners = rotation.apply(corners)

    # Shift to center position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define cube edges (pairs of points)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),  # Bottom square
        (4, 5), (5, 7), (7, 6), (6, 4),  # Top square
        (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges
    ]

    # Create edge traces
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color=color, width=3),
            showlegend=False
        ))

    return edge_traces

def random_color():
    """ Generate a random color in Plotly format """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgb({r},{g},{b})"

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:1]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Generate Interactive 3D Plot in Plotly with Clear Cubes
###############################################################################

fig = go.Figure()

deg_to_rad = np.pi / 180.0
max_deg = 3.0        # Maximum angular variation in degrees
max_side = 0.01     # Maximum cube side length

# Max RPY offsets
max_roll, max_pitch, max_yaw = 10 * deg_to_rad, 5 * deg_to_rad, 50 * deg_to_rad

# Exponential decay rate
lambda_decay = 1.5  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay function for shrinking variation
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # Shrinking angle offsets
        delta_angle = fraction * max_deg * deg_to_rad
        roll_offset = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset = fraction * max_yaw

        # Shrinking cube size
        side_len = fraction * max_side

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Apply small random variations in azimuth and elevation
        az_variation = az + (random.choice([-1, 1]) * delta_angle)
        el_variation = el + (random.choice([-1, 1]) * delta_angle)
        x_new, y_new, z_new = sph2cart(r, az_variation, el_variation)

        # Apply small random variations in roll, pitch, yaw
        roll_var = roll + (random.choice([-1, 1]) * roll_offset)
        pitch_var = pitch + (random.choice([-1, 1]) * pitch_offset)
        yaw_var = yaw + (random.choice([-1, 1]) * yaw_offset)

        rotation_matrix = R.from_euler('xyz', [roll_var, pitch_var, yaw_var])

        if side_len > 1e-9:
            cube_edges = make_cube(x_new, y_new, z_new, rotation_matrix, side_len, random_color())
            for edge in cube_edges:
                fig.add_trace(edge)

# Set figure layout
fig.update_layout(
    title="Interactive Viewpoint Generation with Clearly Visible Cubes",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(projection=dict(type="perspective"))  # Ensures proper 3D depth
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Save interactive figure as HTML
fig.write_html("interactive_viewpoints.html")

# Show interactive plot
fig.show()


In [ ]:
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Function to Create a Cube with Edges in Plotly
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005, color="blue"):
    """ Create a visible 3D cube using Scatter3D edges """
    half = side / 2.0

    # Define cube corners
    corners = np.array([
        [-half, -half, -half], [-half, -half, half],
        [-half,  half, -half], [-half,  half, half],
        [ half, -half, -half], [ half, -half, half],
        [ half,  half, -half], [ half,  half, half]
    ])

    # Apply rotation
    rotated_corners = rotation.apply(corners)

    # Shift to center position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define cube edges (pairs of points)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),  # Bottom square
        (4, 5), (5, 7), (7, 6), (6, 4),  # Top square
        (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges
    ]

    # Create edge traces
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color=color, width=3),
            showlegend=False
        ))

    return edge_traces

def random_color():
    """ Generate a random color in Plotly format """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgb({r},{g},{b})"

###############################################################################
# 2) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 2 demos
    for demo_name in demo_keys[:1]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]  # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 3) Generate Interactive 3D Plot with One Cube Per Trajectory Point
###############################################################################

fig = go.Figure()

cube_size = 0.01  # Fixed cube size

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Compute rotation matrix from RPY values
        rotation_matrix = R.from_euler('xyz', [roll, pitch, yaw])

        # Create cube edges and add to figure
        cube_edges = make_cube(x, y, z, rotation_matrix, cube_size, random_color())
        for edge in cube_edges:
            fig.add_trace(edge)

# Set figure layout
fig.update_layout(
    title="Trajectory Viewpoints with Pose-Aligned Cubes",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(projection=dict(type="perspective"))  # Ensures proper 3D depth
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Save interactive figure as HTML
fig.write_html("trajectory_cubes.html")

# Show interactive plot
fig.show()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """ Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation) """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """ Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z) """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Function to Create a Cube with Edges in Plotly
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005, color="blue"):
    """ Create a visible 3D cube using Scatter3D edges """
    half = side / 2.0

    # Define cube corners
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])

    # Apply rotation
    rotated_corners = rotation.apply(corners)

    # Shift to center position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define cube edges (pairs of points)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),  # Bottom square
        (4, 5), (5, 7), (7, 6), (6, 4),  # Top square
        (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges
    ]

    # Create edge traces
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color=color, width=3),
            showlegend=False
        ))

    return edge_traces

def random_color():
    """ Generate a random color in Plotly format """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgb({r},{g},{b})"

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 1 (or 2) demos for brevity
    for demo_name in demo_keys[:1]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]      # (x, y, z)
            rpy = robot_states[:, 3:6]     # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Generate Interactive 3D Plot in Plotly with Clear Cubes
###############################################################################

fig = go.Figure()

deg_to_rad = np.pi / 180.0
max_deg = 3.0         # Maximum az/el variation in degrees
max_side = 0.01       # Maximum cube side length

# Max RPY offsets (roll, pitch, yaw)
max_roll  = 10 * deg_to_rad
max_pitch =  5 * deg_to_rad
max_yaw   = 50 * deg_to_rad

# Exponential decay rate
lambda_decay = 1.5  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue

    for i, (xyz, rpy) in enumerate(zip(positions, orientations)):
        x, y, z = xyz
        roll, pitch, yaw = rpy

        # Exponential decay factor
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # (A) Shrinking positional offsets
        delta_angle = fraction * max_deg * deg_to_rad  # in radians

        # Convert to spherical
        r, az, el = cart2sph(x, y, z)

        # Blind small random variations in az & el
        az_variation = az + random.choice([-1, 1]) * delta_angle
        el_variation = el + random.choice([-1, 1]) * delta_angle
        x_new, y_new, z_new = sph2cart(r, az_variation, el_variation)

        # (B) Shrinking orientation offsets
        roll_offset  = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset   = fraction * max_yaw

        # Blind small random variations in R, P, Y
        roll_var  = roll  + random.choice([-1, 1]) * roll_offset
        pitch_var = pitch + random.choice([-1, 1]) * pitch_offset
        yaw_var   = yaw   + random.choice([-1, 1]) * yaw_offset

        rotation_matrix = R.from_euler('xyz', [roll_var, pitch_var, yaw_var])

        # (C) Cube size shrinks too
        side_len = fraction * max_side

        if side_len > 1e-9:
            cube_edges = make_cube(x_new, y_new, z_new, rotation_matrix, side_len, random_color())
            for edge in cube_edges:
                fig.add_trace(edge)

# Configure the layout
fig.update_layout(
    title="Interactive Viewpoint Generation (Az/El + R/P/Y Shrinking)",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(projection=dict(type="perspective"))
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Save interactive figure
fig.write_html("interactive_viewpoints.html")

# Show interactive plot
fig.show()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation),
    where:
    - r is distance,
    - az is angle in x-y plane,
    - el is angle from the xy-plane up to z.
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)        # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z).
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Function to Create a Cube with Edges in Plotly
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005, color="blue"):
    """ Create a visible 3D cube using Scatter3D edges """
    half = side / 2.0

    # Define cube corners
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])

    # Apply rotation
    rotated_corners = rotation.apply(corners)

    # Shift to center position
    rotated_corners[:, 0] += cx
    rotated_corners[:, 1] += cy
    rotated_corners[:, 2] += cz

    # Define cube edges (pairs of points)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),  # Bottom square
        (4, 5), (5, 7), (7, 6), (6, 4),  # Top square
        (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges
    ]

    # Create edge traces
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color=color, width=3),
            showlegend=False
        ))

    return edge_traces

def random_color():
    """ Generate a random color in Plotly format """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgb({r},{g},{b})"

###############################################################################
# 3) Load Data: Extract Position and Orientation (RPY)
###############################################################################

file_path = "demo_duck_feb12.hdf5"

positions_list = []
orientations_list = []

with h5py.File(file_path, "r") as f:
    demo_keys = list(f["data"].keys())
    print("Available Demos:", demo_keys)

    # Only take first 1 (or 2) demos for brevity
    for demo_name in demo_keys[:1]:
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
            xyz = robot_states[:, :3]   # (x, y, z)
            rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
            positions_list.append(xyz)
            orientations_list.append(rpy)

###############################################################################
# 4) Generate Interactive 3D Plot (relative to the last point)
###############################################################################

fig = go.Figure()

# Variation settings
deg_to_rad = np.pi / 180.0
max_deg   = 3.0           # Maximum random offset for az/el in degrees
max_side  = 0.01          # Maximum cube side length

# Max RPY offsets
max_roll  = 10 * deg_to_rad
max_pitch =  5 * deg_to_rad
max_yaw   = 5 * deg_to_rad

# Exponential decay rate
lambda_decay = 1.5  

for demo_idx, (positions, orientations) in enumerate(zip(positions_list, orientations_list)):
    n_points = len(positions)
    if n_points < 2:
        continue
    
    # ------------------------
    # GOAL point is the LAST point in the trajectory
    # We will measure everything relative to that goal
    # ------------------------
    goal_pos = positions[-1]          # (x_g, y_g, z_g)
    goal_rpy = orientations[-1]       # (roll_g, pitch_g, yaw_g)

    goal_x, goal_y, goal_z = goal_pos
    goal_roll, goal_pitch, goal_yaw = goal_rpy

    for i in range(n_points):
        # Current point in the trajectory
        x, y, z    = positions[i]
        roll, pitch, yaw = orientations[i]

        # 1) POSitional difference from goal
        dx = x - goal_x
        dy = y - goal_y
        dz = z - goal_z

        # Convert the difference to spherical coordinates
        r, az, el = cart2sph(dx, dy, dz)

        # 2) ORientation difference from goal
        droll  = roll  - goal_roll
        dpitch = pitch - goal_pitch
        dyaw   = yaw   - goal_yaw

        # Exponential decay fraction: bigger i => smaller fraction
        fraction = np.exp(-lambda_decay * (i / (n_points - 1)))

        # ------------------------
        # Random variations in spherical coords (az, el)
        # ------------------------
        delta_angle = fraction * (max_deg * deg_to_rad)
        az_var = az + random.choice([-1, 1]) * delta_angle
        el_var = el + random.choice([-1, 1]) * delta_angle

        # Convert back to Cartesian difference
        dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)

        # Add difference to the goal position
        x_new = goal_x + dx_var
        y_new = goal_y + dy_var
        z_new = goal_z + dz_var

        # ------------------------
        # Random variations in RPY difference
        # ------------------------
        roll_offset  = fraction * max_roll
        pitch_offset = fraction * max_pitch
        yaw_offset   = fraction * max_yaw

        droll_var  = droll  + random.choice([-1, 1]) * roll_offset
        dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
        dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset

        # Compute the new RPY = goal RPY + offset
        roll_new  = goal_roll  + droll_var
        pitch_new = goal_pitch + dpitch_var
        yaw_new   = goal_yaw   + dyaw_var

        # Convert RPY to rotation
        rotation_matrix = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])

        # The cube size also shrinks
        side_len = fraction * max_side
        if side_len < 1e-9:
            continue

        # Create the cube edges, add them
        cube_edges = make_cube(x_new, y_new, z_new, rotation_matrix, side_len, random_color())
        for edge in cube_edges:
            fig.add_trace(edge)

# Configure the layout
fig.update_layout(
    title="Spherical + RPY Variation Relative to GOAL Point",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        camera=dict(projection=dict(type="perspective"))
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Save interactive figure
fig.write_html("interactive_viewpoints.html")

# Show interactive plot
fig.show()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)        # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (for visualization and to generate corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005, color="blue"):
    """ Create a visible 3D cube (Plotly traces) """
    half = side / 2.0
    # Define cube corners in local frame
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    # Rotate and shift corners to world frame
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # Define cube edges (pairs of points for visualization)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),  # Bottom square
        (4, 5), (5, 7), (7, 6), (6, 4),  # Top square
        (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges
    ]
    
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color=color, width=3),
            showlegend=False
        ))
    return edge_traces, rotated_corners  # return both for FoV check

def random_color():
    """ Generate a random Plotly color string """
    r, g, b = np.random.randint(0, 255, 3)
    return f"rgb({r},{g},{b})"

###############################################################################
# 3) Field of View Checker Function and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    """
    Creates and displays a black image with a white dot at the given coordinates.
    """
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)  # short wait to update the window
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if a point is within the camera's field of view.
    
    Parameters:
      camera_pose (tuple): (x, y, z, qx, qy, qz, qw)
      camera_fov (tuple): (horizontal_fov, vertical_fov) in degrees
      point_obj (tuple): (x, y, z) - Object position (goal) in robot base frame
      
    Returns:
      bool: True if the object is within the FOV, False otherwise
    """
    # Extract camera position and quaternion
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform object point into the camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if z_cam <= 0:
        return False  # object is behind the camera
    
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Cube Corner Viewpoints and Evaluate FoV
###############################################################################

def create_dataset():
    # Load trajectory data from HDF5 file
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # For brevity, take only the first demo
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
                xyz = robot_states[:, :3]   # (x, y, z)
                rpy = robot_states[:, 3:6]  # (roll, pitch, yaw)
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure for visualization
    fig = go.Figure()
    
    # Variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 3.0           # Maximum random offset for az/el in degrees
    max_side  = 0.01          # Maximum cube side length

    max_roll  = 10 * deg_to_rad
    max_pitch =  5 * deg_to_rad
    max_yaw   =  5 * deg_to_rad

    lambda_decay = 1.5  # Exponential decay factor
    
    # Define camera FOV (in degrees) for FoV check (e.g., Intel RealSense D435i RGB FOV)
    camera_fov = (69, 42)
    
    # This list will store all generated viewpoints with:
    # - The 6D pose [x, y, z, roll, pitch, yaw]
    # - A Boolean flag indicating if the goal is visible from that viewpoint.
    dataset = []
    
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        
        # Define GOAL as the last point in the trajectory
        goal_pos = positions[-1]          # (x, y, z)
        goal_rpy = orientations[-1]       # (roll, pitch, yaw)
        
        goal_x, goal_y, goal_z = goal_pos
        goal_roll, goal_pitch, goal_yaw = goal_rpy
        
        for i in range(n_points):
            # Get current trajectory point
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            # Compute difference relative to the goal position
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            
            # Orientation difference relative to goal
            droll  = roll  - goal_roll
            dpitch = pitch - goal_pitch
            dyaw   = yaw   - goal_yaw
            
            # Exponential decay for random variation (later points vary less)
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            
            # Apply random variation in spherical coordinates
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            # Compute new position relative to goal
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            # Apply random variation in RPY
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            # New orientation is goal orientation plus variation
            roll_new  = goal_roll  + droll_var
            pitch_new = goal_pitch + dpitch_var
            yaw_new   = goal_yaw   + dyaw_var
            
            # Create rotation object from Euler angles
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            
            # Determine cube side length (shrink with increasing i)
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            # Generate cube visualization and obtain the cube's 8 corner points
            cube_traces, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len, random_color())
            for edge in cube_traces:
                fig.add_trace(edge)
            
            # For each corner of the cube, evaluate the FoV
            # (Each corner viewpoint inherits the cube's orientation)
            quat = rotation_obj.as_quat()  # [qx, qy, qz, qw]
            for j, corner in enumerate(cube_corners):
                # Form a camera pose as a 7-tuple: (x, y, z, qx, qy, qz, qw)
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                # Use the FoV function to check if the goal is visible from this viewpoint.
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                # Save the 6D pose (position + Euler angles) along with the FoV result
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    # (Optional) Save the dataset to a file (here we print its size and can later save as needed)
    print(f"Generated {len(dataset)} viewpoint entries.")

    return dataset
    
    # For example, you can save the dataset as a NumPy file:
    # np.save("viewpoints_dataset.npy", dataset)
    
    # # Configure and show the Plotly figure
    # fig.update_layout(
    #     title="Spherical + RPY Variation Relative to GOAL Point",
    #     scene=dict(
    #         xaxis_title="X",
    #         yaxis_title="Y",
    #         zaxis_title="Z",
    #         camera=dict(projection=dict(type="perspective"))
    #     ),
    #     margin=dict(l=0, r=0, b=0, t=40)
    # )
    # fig.write_html("integrated_viewpoints.html")
    # fig.show()




In [ ]:
dataset = create_dataset()
dataset

In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)        # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (for obtaining the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """ 
    Create cube corners (8 points) given the center and rotation.
    Returns the cube edges (for visualization) and the 8 corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # Create edges for visualization (not used for final scatter points)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if a point is within the camera's field of view.
    camera_pose: (x, y, z, qx, qy, qz, qw)
    camera_fov: (horizontal_fov, vertical_fov) in degrees
    point_obj: (x, y, z) object position
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if z_cam <= 0:
        return False
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints and Plot Color-Coded Points
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure (we will plot only markers)
    fig = go.Figure()
    
    # Variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 3.0           # maximum random offset for azimuth/elevation in degrees
    max_side  = 0.01          # maximum cube side length
    max_roll  = 10 * deg_to_rad
    max_pitch =  5 * deg_to_rad
    max_yaw   =  5 * deg_to_rad
    lambda_decay = 1.5
    
    # Camera FOV for FoV check (in degrees)
    camera_fov = (120, 120)
    
    # Will store each viewpoint (position and 6D pose) and FoV status.
    dataset = []
    
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        
        # Define goal as the last point in the trajectory
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        goal_roll, goal_pitch, goal_yaw = goal_rpy
        
        for i in range(n_points):
            # Get current trajectory point
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            # Compute differences relative to goal
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_roll
            dpitch = pitch - goal_pitch
            dyaw   = yaw   - goal_yaw
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_roll  + droll_var
            pitch_new = goal_pitch + dpitch_var
            yaw_new   = goal_yaw   + dyaw_var
            
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            # Get cube corners (potential viewpoints)
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            
            # Compute quaternion from Euler (for the camera pose)
            quat = rotation_obj.as_quat()  # [qx, qy, qz, qw]
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    # Prepare lists for scatter plot: in-FoV points (green) and out-of-FoV (red)
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot in-FoV points in green
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    
    # Plot out-of-FoV points in red
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    
    # Optionally, mark the goal position
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    # Configure and show the Plotly figure
    fig.update_layout(
        title="Viewpoint Positions: Green = In FoV, Red = Out of FoV",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    
    fig.write_html("colored_viewpoints.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)        # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (for obtaining the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """ 
    Create cube corners (8 points) given the center and rotation.
    Returns the cube edges (for visualization) and the 8 corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # Create edges for visualization (optional)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if a point is within the camera's field of view.
    camera_pose: (x, y, z, qx, qy, qz, qw)
    camera_fov: (horizontal_fov, vertical_fov) in degrees
    point_obj: (x, y, z) object position
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    # Transform the object point into the camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if z_cam <= 0:
        return False
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) "Look-At" Rotation: Compute a rotation so that the camera points toward the goal
###############################################################################

def look_at(cam_pos, target, up=np.array([0, 1, 0])):
    """
    Computes a rotation so that the camera at cam_pos is oriented to look at target.
    The returned rotation is such that the camera's:
      - Z axis (forward) points toward (target - cam_pos),
      - X axis points to the right, and
      - Y axis points upward.
    """
    cam_pos = np.array(cam_pos)
    target = np.array(target)
    forward = target - cam_pos
    norm = np.linalg.norm(forward)
    if norm < 1e-6:
        forward = np.array([0, 0, 1])
    else:
        forward = forward / norm
    # Compute right vector as cross(up, forward)
    right = np.cross(up, forward)
    norm_r = np.linalg.norm(right)
    if norm_r < 1e-6:
        # Choose an alternate up vector if up is parallel to forward
        up = np.array([1, 0, 0])
        right = np.cross(up, forward)
        right = right / np.linalg.norm(right)
    else:
        right = right / norm_r
    true_up = np.cross(forward, right)
    # Construct rotation matrix; columns are (right, true_up, forward)
    R_cam = np.column_stack((right, true_up, forward))
    return R.from_matrix(R_cam)

###############################################################################
# 5) Main Integration: Generate Viewpoints and Plot Color-Coded Points
###############################################################################

def main():
    # Set to True to force camera orientations to "look at" the goal.
    # This increases the chance that the goal will fall within the FoV.
    use_lookat = True

    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure (we will plot only markers)
    fig = go.Figure()
    
    # Variation settings (position and orientation noise)
    deg_to_rad = np.pi / 180.0
    max_deg   = 3.0           # max offset for azimuth/elevation (degrees)
    max_side  = 0.01          # max cube side length
    max_roll  = 10 * deg_to_rad
    max_pitch =  5 * deg_to_rad
    max_yaw   =  5 * deg_to_rad
    lambda_decay = 1.5
    
    # Camera FOV for FoV check (in degrees)
    camera_fov = (69, 42)
    
    # Will store each viewpoint and its FoV status.
    dataset = []
    
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        
        # Define goal as the last point in the trajectory
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        for i in range(n_points):
            # Get current trajectory point and orientation
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            # Compute difference relative to goal
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            # Use the random orientation for the cube (for position generation)
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            # Obtain cube corners (candidate camera positions)
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            
            for j, corner in enumerate(cube_corners):
                if use_lookat:
                    # Compute a camera rotation that looks from the corner to the goal.
                    cam_rot = look_at(corner, goal_pos)
                    quat = cam_rot.as_quat()
                else:
                    quat = rotation_obj.as_quat()
                # Build camera pose: position + quaternion
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    # Separate viewpoint positions by FoV status
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot in-FoV points in green
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    
    # Plot out-of-FoV points in red
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    
    # Mark the goal position in blue
    goal_x_val, goal_y_val, goal_z_val = goal_x, goal_y, goal_z
    fig.add_trace(go.Scatter3d(
        x=[goal_x_val],
        y=[goal_y_val],
        z=[goal_z_val],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Viewpoint Positions: Green = In FoV, Red = Out of FoV",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("colored_viewpoints.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)        # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (for obtaining the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx,cy,cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # For visualization, create cube edge traces (optional)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object point (point_obj) is within the camera's field of view.
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) - object position in robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform the object into the camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if z_cam <= 0:
        return False  # Object is behind the camera
    
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints with Narrow Variation
###############################################################################

def main():
    # Load trajectory data from HDF5 file
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure for visualization
    fig = go.Figure()
    
    # Variation settings (narrowed random variation)
    deg_to_rad = np.pi / 180.0
    max_deg   = 1.0           # Narrow variation: up to 1 degree change in azimuth/elevation
    max_side  = 0.01          # Cube side length remains the same
    max_roll  = 2 * deg_to_rad    # Narrow orientation variation: up to 2 degrees
    max_pitch = 1 * deg_to_rad    # up to 1 degree
    max_yaw   = 1 * deg_to_rad    # up to 1 degree
    lambda_decay = 1.5
    
    # Widened camera FoV (in degrees) to further help with goal visibility
    camera_fov = (90, 60)
    
    # List to store each generated viewpoint with its pose and FoV result
    dataset = []
    
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        
        # Define goal as the last point in the trajectory
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        for i in range(n_points):
            # Current trajectory point and orientation
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            # Compute differences relative to the goal
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            # Apply narrow random variation with exponential decay
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            # Compute new position relative to the goal
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            # Apply narrow orientation variation
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            # Construct rotation from generated Euler angles
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            # Generate the cube corners (candidate camera viewpoints)
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            
            # Use the cube's realistic (random) orientation for each corner viewpoint
            quat = rotation_obj.as_quat()
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    # Separate viewpoint positions for plotting
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot in-FoV points in green and out-of-FoV points in red
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    
    # Mark the goal position in blue
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Viewpoint Positions with Narrow Variation: Green = In FoV, Red = Out of FoV",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    # fig.write_html("colored_viewpoints_narrow_variation.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (to obtain the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx,cy,cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # (Optional) Create edge traces for visualization
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) in robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    # If the object is nearly co-located with the camera, consider it visible.
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform object into camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if z_cam <= 0:
        return False  # Object is behind the camera
    
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints with Narrow Variation and Draw Camera Direction
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure
    fig = go.Figure()
    
    # Narrow variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 1.0            # up to 1° variation in azimuth/elevation
    max_side  = 0.01           # cube side length constant
    max_roll  = 2 * deg_to_rad   # up to 2° roll variation
    max_pitch = 1 * deg_to_rad   # up to 1° pitch variation
    max_yaw   = 1 * deg_to_rad   # up to 1° yaw variation
    lambda_decay = 1.5
    
    # Use a wide FoV
    camera_fov = (90, 60)
    
    # List to store viewpoint data
    dataset = []
    
    # For diagnostics, check the FoV of the demonstration pose (last trajectory point)
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        # Use the demonstration pose directly (no variation)
        demo_rot = R.from_euler('xyz', goal_rpy)
        demo_quat = demo_rot.as_quat()
        demo_pose = (goal_x, goal_y, goal_z, demo_quat[0], demo_quat[1], demo_quat[2], demo_quat[3])
        demo_in_fov = is_inFOV(demo_pose, camera_fov, goal_pos, show_img=False)
        print(f"Demonstration pose FoV check (should be True): {demo_in_fov}")
        
        # --- Draw the camera direction for the demonstration pose ---
        # Assume camera looks along its positive z-axis.
        arrow_length = 0.05  # Adjust for desired arrow length
        demo_cam_pos = np.array([goal_x, goal_y, goal_z])
        demo_cam_rot = R.from_quat(demo_quat).as_matrix()
        forward_vector = demo_cam_rot @ np.array([0, 0, arrow_length])
        demo_arrow_end = demo_cam_pos + forward_vector
        fig.add_trace(go.Scatter3d(
            x=[demo_cam_pos[0], demo_arrow_end[0]],
            y=[demo_cam_pos[1], demo_arrow_end[1]],
            z=[demo_cam_pos[2], demo_arrow_end[2]],
            mode='lines+markers',
            marker=dict(size=4, color='orange'),
            line=dict(color='orange', width=4),
            name='Demo Camera Direction'
        ))
        
        # Generate candidate viewpoints along the trajectory
        for i in range(n_points):
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            # Compute difference relative to goal
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            quat = rotation_obj.as_quat()
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints: green if goal is in FoV, red otherwise
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints with Demo Camera Direction (Orange Arrow)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("colored_viewpoints_with_camera_direction.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (to obtain the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx,cy,cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # (Optional) Create edge traces for visualization
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) in robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform object into camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if z_cam <= 0:
        return False
    
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints and Draw Camera Directions for Each Trajectory Point
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure
    fig = go.Figure()
    
    # Narrow variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 1.0            # up to 1° variation in azimuth/elevation
    max_side  = 0.01           # cube side length constant
    max_roll  = 2 * deg_to_rad   # up to 2° roll variation
    max_pitch = 1 * deg_to_rad   # up to 1° pitch variation
    max_yaw   = 1 * deg_to_rad   # up to 1° yaw variation
    lambda_decay = 1.5
    
    # Use a wide FoV
    camera_fov = (90, 60)
    
    # Arrow length for drawing camera direction
    arrow_length = 0.05
    
    # Draw camera direction arrows for each trajectory point
    # (Using the original demonstration trajectory poses)
    for positions, orientations in zip(positions_list, orientations_list):
        for idx, (pos, rpy) in enumerate(zip(positions, orientations)):
            cam_rot = R.from_euler('xyz', rpy).as_matrix()
            # Assume the camera looks along its positive z-axis in its own frame.
            forward = cam_rot @ np.array([0, 0, arrow_length])
            arrow_end = pos + forward
            # Only show one legend entry for the first arrow.
            fig.add_trace(go.Scatter3d(
                x=[pos[0], arrow_end[0]],
                y=[pos[1], arrow_end[1]],
                z=[pos[2], arrow_end[2]],
                mode='lines',
                line=dict(color='orange', width=3),
                name='Camera Direction' if idx == 0 else None
            ))
    
    # List to store candidate viewpoint data
    dataset = []
    
    # For diagnostics, check the FoV of the demonstration (last trajectory) pose
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        demo_rot = R.from_euler('xyz', goal_rpy)
        demo_quat = demo_rot.as_quat()
        demo_pose = (goal_x, goal_y, goal_z,
                     demo_quat[0], demo_quat[1], demo_quat[2], demo_quat[3])
        demo_in_fov = is_inFOV(demo_pose, camera_fov, goal_pos, show_img=False)
        print(f"Demonstration pose FoV check (should be True): {demo_in_fov}")
        
        # Generate candidate viewpoints along the trajectory with narrow variation
        for i in range(n_points):
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            quat = rotation_obj.as_quat()
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints: green if goal is in FoV, red otherwise.
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints with Camera Directions from Trajectory Points",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("colored_viewpoints_with_trajectory_camera_directions.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (to obtain the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx,cy,cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # (Optional) Create edge traces for visualization
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) in robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    # If the object is nearly at the camera, consider it visible.
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform object into camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if z_cam <= 0:
        return False  # Object is behind the camera
    
    h_angle = np.arctan2(x_cam, z_cam)
    v_angle = np.arctan2(y_cam, z_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints and Draw Camera Directions
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure
    fig = go.Figure()
    
    # Narrow variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 1.0            # up to 1° variation in azimuth/elevation
    max_side  = 0.01           # cube side length constant
    max_roll  = 2 * deg_to_rad   # up to 2° roll variation
    max_pitch = 1 * deg_to_rad   # up to 1° pitch variation
    max_yaw   = 1 * deg_to_rad   # up to 1° yaw variation
    lambda_decay = 1.5
    
    # Use a wide FoV
    camera_fov = (90, 60)
    
    # Define arrow length for drawing camera direction
    arrow_length = 0.05
    
    # Draw camera direction arrows for each trajectory point
    # Now we assume the camera's optical axis (forward) is along the x-axis.
    for positions, orientations in zip(positions_list, orientations_list):
        for idx, (pos, rpy) in enumerate(zip(positions, orientations)):
            # Compute rotation matrix from Euler angles (using 'xyz' convention)
            cam_rot = R.from_euler('xyz', rpy).as_matrix()
            # The camera's forward vector is now assumed to be along the x-axis.
            forward = cam_rot @ np.array([arrow_length, 0, 0])
            arrow_end = pos + forward
            # Draw an arrow (orange line) for each point.
            fig.add_trace(go.Scatter3d(
                x=[pos[0], arrow_end[0]],
                y=[pos[1], arrow_end[1]],
                z=[pos[2], arrow_end[2]],
                mode='lines',
                line=dict(color='orange', width=3),
                name='Camera Direction' if idx == 0 else None
            ))
    
    # List to store candidate viewpoint data
    dataset = []
    
    # For diagnostics, check the FoV of the demonstration (last trajectory) pose
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        demo_rot = R.from_euler('xyz', goal_rpy)
        demo_quat = demo_rot.as_quat()
        demo_pose = (goal_x, goal_y, goal_z,
                     demo_quat[0], demo_quat[1], demo_quat[2], demo_quat[3])
        demo_in_fov = is_inFOV(demo_pose, camera_fov, goal_pos, show_img=False)
        print(f"Demonstration pose FoV check (should be True): {demo_in_fov}")
        
        # Generate candidate viewpoints along the trajectory with narrow variation
        for i in range(n_points):
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            quat = rotation_obj.as_quat()
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints: green if goal is in FoV, red otherwise.
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints with Camera Directions from Trajectory Points",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("colored_viewpoints_with_trajectory_camera_directions.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (to obtain the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx,cy,cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # Optional: create edge traces for visualization
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    This version assumes the camera's optical (forward) axis is the positive x-axis.
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) in the robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    # If the object is nearly at the camera, consider it visible.
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform the object into the camera frame.
    # Now the camera's forward axis is x.
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if x_cam <= 0:
        return False  # Object is behind the camera
    
    # Compute angles relative to the x-axis.
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints and Draw Camera Directions
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure
    fig = go.Figure()
    
    # Narrow variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 1.0            # up to 1° variation in azimuth/elevation
    max_side  = 0.01           # cube side length constant
    max_roll  = 2 * deg_to_rad   # up to 2° roll variation
    max_pitch = 1 * deg_to_rad   # up to 1° pitch variation
    max_yaw   = 1 * deg_to_rad   # up to 1° yaw variation
    lambda_decay = 1.5
    
    # Use a wide FoV
    camera_fov = (150, 150)
    
    # Define arrow length for drawing camera direction
    arrow_length = 0.05
    
    # Draw camera direction arrows for each trajectory point
    # Now we assume the camera's optical axis is along the x-axis.
    for positions, orientations in zip(positions_list, orientations_list):
        for idx, (pos, rpy) in enumerate(zip(positions, orientations)):
            cam_rot = R.from_euler('xyz', rpy).as_matrix()
            # Forward vector now along the x-axis.
            forward = cam_rot @ np.array([arrow_length, 0, 0])
            arrow_end = pos + forward
            fig.add_trace(go.Scatter3d(
                x=[pos[0], arrow_end[0]],
                y=[pos[1], arrow_end[1]],
                z=[pos[2], arrow_end[2]],
                mode='lines',
                line=dict(color='orange', width=3),
                name='Camera Direction' if idx == 0 else None
            ))
    
    # List to store candidate viewpoint data
    dataset = []
    
    # For diagnostics, check the FoV of the demonstration (last trajectory) pose
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        demo_rot = R.from_euler('xyz', goal_rpy)
        demo_quat = demo_rot.as_quat()
        demo_pose = (goal_x, goal_y, goal_z,
                     demo_quat[0], demo_quat[1], demo_quat[2], demo_quat[3])
        demo_in_fov = is_inFOV(demo_pose, camera_fov, goal_pos, show_img=False)
        print(f"Demonstration pose FoV check (should be True): {demo_in_fov}")
        
        # Generate candidate viewpoints along the trajectory with narrow variation
        for i in range(n_points):
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            rotation_obj = R.from_euler('xyz', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            quat = rotation_obj.as_quat()
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints: green if goal is in FoV, red otherwise.
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints with Correct Camera Directions (Optical Axis = X)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("colored_viewpoints_corrected_camera_direction.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (to obtain the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx, cy, cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # Create edge traces for visualization (optional)
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    This version now assumes the camera's optical (forward) axis is the positive x-axis.
    (Using the 'zyx' conversion order for RPY.)
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) in the robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    # Use 'zyx' conversion order for the Euler angles.
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()  # Assuming the quaternions are computed from R.from_euler('zyx', ...)
    
    # Transform the object into the camera frame (optical axis is now x)
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if x_cam <= 0:
        return False  # Object is behind the camera
    
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov / 2 and abs(v_angle) <= v_fov / 2

###############################################################################
# 4) Main Integration: Generate Viewpoints and Draw Camera Directions
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:2]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                xyz = robot_states[:, :3]
                rpy = robot_states[:, 3:6]
                positions_list.append(xyz)
                orientations_list.append(rpy)
    
    # Setup Plotly figure
    fig = go.Figure()
    
    # Narrow variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 8.0            # up to 1° variation in azimuth/elevation
    max_side  = 0.01           # cube side length constant
    max_roll  = 5 * deg_to_rad   # up to 2° roll variation
    max_pitch = 5 * deg_to_rad   # up to 1° pitch variation
    max_yaw   = 5 * deg_to_rad   # up to 1° yaw variation
    lambda_decay = 1.5
    
    # Use a wide FoV
    camera_fov = (90, 90)
    
    # Define arrow length for drawing camera direction
    arrow_length = 0.05
    
    # Draw camera direction arrows for each trajectory point
    # Here we use the 'zyx' conversion order.
    # for positions, orientations in zip(positions_list, orientations_list):
    #     for idx, (pos, rpy) in enumerate(zip(positions, orientations)):
    #         # Use 'zyx' conversion to obtain the rotation matrix.
    #         cam_rot = R.from_euler('zyx', rpy).as_matrix()
    #         # Assume the camera's forward (optical) axis is along the x-axis.
    #         forward = cam_rot @ np.array([arrow_length, 0, 0])
    #         arrow_end = pos + forward
    #         fig.add_trace(go.Scatter3d(
    #             x=[pos[0], arrow_end[0]],
    #             y=[pos[1], arrow_end[1]],
    #             z=[pos[2], arrow_end[2]],
    #             mode='lines',
    #             line=dict(color='orange', width=3),
    #             name='Camera Direction' if idx == 0 else None
    #         ))
    
    # List to store candidate viewpoint data
    dataset = []
    
    # For diagnostics, check the FoV of the demonstration (last trajectory) pose
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        # Compute demonstration pose using 'zyx' order.
        demo_rot = R.from_euler('zyx', goal_rpy)
        demo_quat = demo_rot.as_quat()
        demo_pose = (goal_x, goal_y, goal_z,
                     demo_quat[0], demo_quat[1], demo_quat[2], demo_quat[3])
        demo_in_fov = is_inFOV(demo_pose, camera_fov, goal_pos, show_img=False)
        print(f"Demonstration pose FoV check (should be True): {demo_in_fov}")
        
        # Generate candidate viewpoints along the trajectory with narrow variation
        for i in range(n_points):
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r, az_var, el_var)
            
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            # Use the 'zyx' conversion order for the candidate pose.
            rotation_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            quat = rotation_obj.as_quat()
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos, show_img=False)
                viewpoint = {
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                }
                dataset.append(viewpoint)
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints: green if goal is in FoV, red otherwise.
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints with Corrected Camera Directions (Using 'zyx' Order)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("colored_viewpoints_corrected_camera_direction.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (to obtain the corner viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Create cube corners (8 points) given the center (cx, cy, cz) and rotation.
    Returns:
      - edge_traces: list of Plotly traces for visualization (optional)
      - rotated_corners: the 8 corner points (Nx3 array)
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half], [-half, -half,  half],
        [-half,  half, -half], [-half,  half,  half],
        [ half, -half, -half], [ half, -half,  half],
        [ half,  half, -half], [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    
    # (Optional) Create edge traces for visualization
    edges = [
        (0, 1), (1, 3), (3, 2), (2, 0),
        (4, 5), (5, 7), (7, 6), (6, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    edge_traces = []
    for e in edges:
        edge_traces.append(go.Scatter3d(
            x=[rotated_corners[e[0], 0], rotated_corners[e[1], 0]],
            y=[rotated_corners[e[0], 1], rotated_corners[e[1], 1]],
            z=[rotated_corners[e[0], 2], rotated_corners[e[1], 2]],
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return edge_traces, rotated_corners

###############################################################################
# 3) Field of View Checker and Helpers
###############################################################################

def show_object_on_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    cv2.imshow("Object on Image Frame", image)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

def is_inFOV(camera_pose, camera_fov, point_obj, show_img=False, img_width=640, img_height=480):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    This version now assumes the camera's optical (forward) axis is the positive x-axis.
    (Using the 'zyx' conversion order for RPY.)
    
    Parameters:
      camera_pose: (x, y, z, qx, qy, qz, qw)
      camera_fov: (horizontal_fov, vertical_fov) in degrees
      point_obj: (x, y, z) in the robot base frame
      
    Returns:
      bool: True if object is in FoV, False otherwise.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    
    # Transform the object into the camera frame (optical axis is now x)
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    
    if x_cam <= 0:
        return False  # Object is behind the camera
    
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    
    if show_img:
        img_x = (h_angle + h_fov / 2) / h_fov * img_width
        img_y = (v_angle + v_fov / 2) / v_fov * img_height
        show_object_on_image_frame(img_width, img_height, (int(img_x), int(img_y)))
        
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Main Integration: Generate Multiple Candidate Viewpoints and Plot by FoV
###############################################################################

def main():
    # Load trajectory data
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Here we load the first two demos for more data.
        for demo_name in demo_keys[:2]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Setup Plotly figure
    fig = go.Figure()
    
    # Variation settings (wider ranges)
    deg_to_rad = np.pi / 180.0
    max_deg   = 8.0            # up to 8° variation in spherical coordinates
    max_side  = 0.01           # cube side length constant
    max_roll  = 5 * deg_to_rad   # up to 5° roll variation
    max_pitch = 5 * deg_to_rad   # up to 5° pitch variation
    max_yaw   = 5 * deg_to_rad   # up to 5° yaw variation
    lambda_decay = 1.5
    # Use a wide FoV
    camera_fov = (90, 90)
    
    # Set the number of samples per trajectory point
    num_samples = 10
    
    dataset = []
    
    # For each demonstration trajectory
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]
        goal_rpy = orientations[-1]
        goal_x, goal_y, goal_z = goal_pos
        
        # Generate multiple candidate viewpoints for each trajectory point.
        for i in range(n_points):
            # For each trajectory point, sample multiple candidate variations.
            for sample in range(num_samples):
                x, y, z = positions[i]
                roll, pitch, yaw = orientations[i]
                
                # Compute relative difference to the goal
                dx = x - goal_x
                dy = y - goal_y
                dz = z - goal_z
                r_val, az, el = cart2sph(dx, dy, dz)
                droll  = roll  - goal_rpy[0]
                dpitch = pitch - goal_rpy[1]
                dyaw   = yaw   - goal_rpy[2]
                
                fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
                delta_angle = fraction * (max_deg * deg_to_rad)
                az_var = az + random.choice([-1, 1]) * delta_angle
                el_var = el + random.choice([-1, 1]) * delta_angle
                dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
                
                x_new = goal_x + dx_var
                y_new = goal_y + dy_var
                z_new = goal_z + dz_var
                
                roll_offset  = fraction * max_roll
                pitch_offset = fraction * max_pitch
                yaw_offset   = fraction * max_yaw
                droll_var  = droll  + random.choice([-1, 1]) * roll_offset
                dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
                dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
                
                roll_new  = goal_rpy[0] + droll_var
                pitch_new = goal_rpy[1] + dpitch_var
                yaw_new   = goal_rpy[2] + dyaw_var
                
                # Use 'zyx' conversion order for the candidate pose.
                rotation_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
                side_len = fraction * max_side
                if side_len < 1e-9:
                    continue
                
                # Generate the 8 cube corners as candidate viewpoints.
                _, cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
                quat = rotation_obj.as_quat()
                for j, corner in enumerate(cube_corners):
                    cam_pose = (corner[0], corner[1], corner[2],
                                quat[0], quat[1], quat[2], quat[3])
                    in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                    dataset.append({
                        "trajectory_idx": i,
                        "sample": sample,
                        "corner_idx": j,
                        "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                        "in_fov": in_fov
                    })
    
    print(f"Generated {len(dataset)} viewpoint entries.")
    
    # Separate candidate viewpoints for visualization
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints: green if goal is in FoV, red otherwise.
    fig.add_trace(go.Scatter3d(
        x=in_fov_x,
        y=in_fov_y,
        z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x,
        y=out_fov_y,
        z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    # Mark the goal position with a blue marker.
    fig.add_trace(go.Scatter3d(
        x=[goal_x],
        y=[goal_y],
        z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints (Multiple Samples per Trajectory Point)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z"
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("viewpoints_by_fov_multiple_samples.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Generate the 8 corner points of a cube.
    
    Parameters:
      cx, cy, cz: Center position of the cube.
      rotation: A scipy Rotation object (using our updated 'zyx' convention).
      side: The side length of the cube.
      
    Returns:
      rotated_corners: An (8 x 3) array containing the world coordinates
                       of the cube's corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

###############################################################################
# 3) Field of View Checker
###############################################################################

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    Assumptions for our case:
      - camera_pose is a 7-tuple: (x, y, z, qx, qy, qz, qw).
      - The camera’s optical (forward) axis is along the positive x‑axis.
      - RPY values are interpreted using the 'zyx' (yaw–pitch–roll) convention.
    
    Returns:
      True if the goal is within the FoV, else False.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True

    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    # Transform the goal (object) into the camera frame
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:  # Object is behind the camera
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Main Integration: Generate Candidate Viewpoints from Cube Corners
###############################################################################

def main():
    # Load trajectory data from HDF5 file
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # For brevity, take only the first demo
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 10.0            # Maximum random offset in spherical angles (degrees)
    max_side  = 0.1           # Cube side length
    max_roll  = 50 * deg_to_rad
    max_pitch = 40 * deg_to_rad
    max_yaw   = 40 * deg_to_rad
    lambda_decay = 2
    camera_fov = (90, 90)      # Wide FoV (horizontal, vertical)
    
    dataset = []
    
    # The goal is defined as the last point in the trajectory.
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]        # (x, y, z)
        goal_rpy = orientations[-1]     # (roll, pitch, yaw)
        goal_x, goal_y, goal_z = goal_pos
        
        for i in range(n_points):
            # Get current trajectory point pose
            x, y, z = positions[i]
            roll, pitch, yaw = orientations[i]
            
            # Compute difference relative to the goal
            dx = x - goal_x
            dy = y - goal_y
            dz = z - goal_z
            r_val, az, el = cart2sph(dx, dy, dz)
            droll  = roll  - goal_rpy[0]
            dpitch = pitch - goal_rpy[1]
            dyaw   = yaw   - goal_rpy[2]
            
            # Exponential decay so that variation decreases closer to the goal
            fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
            
            # Random variation in spherical coordinates
            delta_angle = fraction * (max_deg * deg_to_rad)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
            
            # New candidate cube center (position)
            x_new = goal_x + dx_var
            y_new = goal_y + dy_var
            z_new = goal_z + dz_var
            
            # Random variation in orientation
            roll_offset  = fraction * max_roll
            pitch_offset = fraction * max_pitch
            yaw_offset   = fraction * max_yaw
            droll_var  = droll  + random.choice([-1, 1]) * roll_offset
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
            dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
            
            # New candidate Euler angles (using our goal-relative offset)
            roll_new  = goal_rpy[0] + droll_var
            pitch_new = goal_rpy[1] + dpitch_var
            yaw_new   = goal_rpy[2] + dyaw_var
            
            # Use the 'zyx' conversion order (yaw-pitch-roll) to compute rotation
            rotation_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
            side_len = fraction * max_side
            if side_len < 1e-9:
                continue
            
            # Generate the 8 corner points of the cube
            cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
            quat = rotation_obj.as_quat()
            
            # Each corner becomes a candidate viewpoint with the cube's orientation.
            for j, corner in enumerate(cube_corners):
                cam_pose = (corner[0], corner[1], corner[2],
                            quat[0], quat[1], quat[2], quat[3])
                # Use the FoV checker to decide if the goal is visible from this candidate.
                in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                dataset.append({
                    "trajectory_idx": i,
                    "corner_idx": j,
                    "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                    "in_fov": in_fov
                })
    
    print(f"Generated {len(dataset)} candidate viewpoints.")
    
    # Visualization: plot candidate viewpoints
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=in_fov_x, y=in_fov_y, z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x, y=out_fov_y, z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    # Mark the goal position
    fig.add_trace(go.Scatter3d(
        x=[goal_x], y=[goal_y], z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints from Cube Corners (Using 'zyx' Convention)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("candidate_viewpoints_from_cube_corners.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)  # [-pi, pi]
    el = np.arctan2(z, np.sqrt(x**2 + y**2))  # [-pi/2, pi/2]
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Generate the 8 corner points of a cube.
    
    Parameters:
      cx, cy, cz: Center of the cube.
      rotation: A scipy Rotation object computed with 'zyx' convention.
      side: Cube side length.
      
    Returns:
      rotated_corners: An (8 x 3) array of the world coordinates of the cube's corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

###############################################################################
# 3) Field of View Checker and Image Generation Helpers
###############################################################################

def generate_image_frame(img_width, img_height, point_coord, dot_dia=10):
    """
    Generate a black image of given dimensions with a white dot drawn at point_coord.
    Returns the image as a numpy array.
    """
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia // 2, (255, 255, 255), -1)
    return image

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    """
    Generate an image representation of the camera view, by projecting the object (point_obj)
    into the image frame.
    
    Assumes:
      - camera_pose: (x, y, z, qx, qy, qz, qw)
      - The camera’s forward axis is the positive x-axis.
      - RPY are interpreted with 'zyx' order.
    
    Returns a numpy array representing the image.
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    # Transform the object into the camera frame.
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    # Compute horizontal and vertical angles relative to the x-axis.
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    # Map angles to image coordinates.
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    return generate_image_frame(img_width, img_height, (img_x, img_y), dot_dia)

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if an object (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose is (x, y, z, qx, qy, qz, qw)
      - The camera's optical (forward) axis is the positive x-axis.
      - RPY are interpreted using the 'zyx' (yaw-pitch-roll) convention.
    
    Returns True if the object is within the FoV, else False.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Main Integration: Generate Candidate Viewpoints from Cube Corners and Store Images
###############################################################################

def main():
    # Load trajectory data from HDF5 file
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # For brevity, use the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Variation settings
    deg_to_rad = np.pi / 180.0
    max_deg   = 5.0            # max random offset in spherical angles (degrees)
    max_side  = 0.1           # cube side length
    max_roll  = 50 * deg_to_rad
    max_pitch = 10 * deg_to_rad
    max_yaw   = 10 * deg_to_rad
    lambda_decay = 1.5
    camera_fov = (80, 80)      # Wide FoV (horizontal, vertical)
    # Image dimensions for camera view simulation:
    img_width, img_height, dot_dia = 640, 480, 10
    # Number of samples per trajectory point
    num_samples = 10
    
    dataset = []
    
    # Define the goal as the last point in the trajectory.
    for positions, orientations in zip(positions_list, orientations_list):
        n_points = len(positions)
        if n_points < 2:
            continue
        goal_pos = positions[-1]       # (x, y, z)
        goal_rpy = orientations[-1]    # (roll, pitch, yaw)
        goal_x, goal_y, goal_z = goal_pos
        
        for i in range(n_points):
            # For each trajectory point, generate multiple candidate variations.
            for sample in range(num_samples):
                x, y, z = positions[i]
                roll, pitch, yaw = orientations[i]
                
                # Compute difference relative to goal.
                dx = x - goal_x
                dy = y - goal_y
                dz = z - goal_z
                r_val, az, el = cart2sph(dx, dy, dz)
                droll  = roll  - goal_rpy[0]
                dpitch = pitch - goal_rpy[1]
                dyaw   = yaw   - goal_rpy[2]
                
                fraction = np.exp(-lambda_decay * (i / (n_points - 1)))
                delta_angle = fraction * (max_deg * deg_to_rad)
                az_var = az + random.choice([-1, 1]) * delta_angle
                el_var = el + random.choice([-1, 1]) * delta_angle
                dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
                
                # New candidate cube center.
                x_new = goal_x + dx_var
                y_new = goal_y + dy_var
                z_new = goal_z + dz_var
                
                # Random variation in orientation.
                roll_offset  = fraction * max_roll
                pitch_offset = fraction * max_pitch
                yaw_offset   = fraction * max_yaw
                droll_var  = droll  + random.choice([-1, 1]) * roll_offset
                dpitch_var = dpitch + random.choice([-1, 1]) * pitch_offset
                dyaw_var   = dyaw   + random.choice([-1, 1]) * yaw_offset
                
                roll_new  = goal_rpy[0] + droll_var
                pitch_new = goal_rpy[1] + dpitch_var
                yaw_new   = goal_rpy[2] + dyaw_var
                
                # Use 'zyx' conversion order (yaw-pitch-roll) to compute rotation.
                rotation_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
                side_len = fraction * max_side
                if side_len < 1e-9:
                    continue
                
                # Generate cube corners as candidate viewpoints.
                cube_corners = make_cube(x_new, y_new, z_new, rotation_obj, side_len)
                quat = rotation_obj.as_quat()
                for j, corner in enumerate(cube_corners):
                    cam_pose = (corner[0], corner[1], corner[2],
                                quat[0], quat[1], quat[2], quat[3])
                    in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                    # Generate the simulated camera image (projection of the goal)
                    cam_image = generate_camera_image(cam_pose, camera_fov, goal_pos,
                                                      img_width, img_height, dot_dia)
                    dataset.append({
                        "trajectory_idx": i,
                        "sample": sample,
                        "corner_idx": j,
                        "pose_6d": [corner[0], corner[1], corner[2], roll_new, pitch_new, yaw_new],
                        "in_fov": in_fov,
                        "image": cam_image  # store the generated image (as a numpy array)
                    })
    
    print(f"Generated {len(dataset)} candidate viewpoints.")
    
    # Separate candidate viewpoints by FoV result for visualization.
    in_fov_x, in_fov_y, in_fov_z = [], [], []
    out_fov_x, out_fov_y, out_fov_z = [], [], []
    for vp in dataset:
        x_pt, y_pt, z_pt = vp["pose_6d"][:3]
        if vp["in_fov"]:
            in_fov_x.append(x_pt)
            in_fov_y.append(y_pt)
            in_fov_z.append(z_pt)
        else:
            out_fov_x.append(x_pt)
            out_fov_y.append(y_pt)
            out_fov_z.append(z_pt)
    
    # Plot candidate viewpoints in 3D.
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=in_fov_x, y=in_fov_y, z=in_fov_z,
        mode='markers',
        marker=dict(size=4, color='green'),
        name='In FoV'
    ))
    fig.add_trace(go.Scatter3d(
        x=out_fov_x, y=out_fov_y, z=out_fov_z,
        mode='markers',
        marker=dict(size=4, color='red'),
        name='Out of FoV'
    ))
    # Mark the goal position.
    fig.add_trace(go.Scatter3d(
        x=[goal_x], y=[goal_y], z=[goal_z],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Candidate Viewpoints from Cube Corners (Images Stored in Dataset)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(projection=dict(type="perspective"))
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    # fig.write_html("candidate_viewpoints_from_cube_corners.html")
    fig.show()
    
    # Randomly sample a few candidate viewpoints and display their generated images with FoV flag.
    sample_candidates = random.sample(dataset, 10)
    for idx, cand in enumerate(sample_candidates):
        img = cand["image"]
        fov_flag = cand["in_fov"]
        plt.figure()
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Candidate {idx} (FoV: {fov_flag})")
        plt.axis('off')
        plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Generate the 8 corner points of a cube.
    
    Parameters:
      cx, cy, cz: Center of the cube.
      rotation: A scipy Rotation object computed using the 'zyx' convention.
      side: Cube side length.
      
    Returns:
      An (8 x 3) numpy array of world coordinates for the cube corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker and Image Generation Helper
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    """
    Generate a synthetic camera image showing where the goal projects.
    The image is black with a white dot at the projected position.
    
    Assumes:
      - camera_pose is (x, y, z, qx, qy, qz, qw)
      - The camera’s forward (optical) axis is the positive x-axis.
      - RPY are interpreted with 'zyx' order.
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    # Compute angles relative to x-axis.
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    # Map angles to image coordinates.
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose is (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x-axis.
      - RPY are interpreted using the 'zyx' (yaw-pitch-roll) convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj)-cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate pose (x, y, z, roll, pitch, yaw) into a 7-tuple:
    (x, y, z, qx, qy, qz, qw), using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

# Map parameter names to indices in our 6D pose.
param_to_index = {'x': 0, 'y': 1, 'z': 2, 'roll': 3, 'pitch': 4, 'yaw': 5}

def get_allowed_variation(base_pose, param, step, max_range, goal, camera_fov):
    """
    Starting from base_pose (a list of 6 values: [x,y,z,roll,pitch,yaw]),
    vary the specified parameter (one of 'x', 'y', 'z', 'roll', 'pitch', 'yaw')
    in both positive and negative directions (in increments of step) until the
    goal is no longer in the camera's field of view. Returns (min_delta, max_delta)
    where these deltas are relative to base_pose.
    """
    idx = param_to_index[param]
    
    # Test positive direction.
    delta = 0.0
    pos_valid = 0.0
    while abs(delta) <= max_range:
        candidate = base_pose.copy()
        candidate[idx] += delta
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            pos_valid = delta
            delta += step
        else:
            break
    
    # Test negative direction.
    delta = 0.0
    neg_valid = 0.0
    while abs(delta) <= max_range:
        candidate = base_pose.copy()
        candidate[idx] -= delta
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            neg_valid = -delta
            delta += step
        else:
            break
    
    return neg_valid, pos_valid

def synthesize_trajectory(base_pose, goal, camera_fov, params, step_dict, max_range_dict):
    """
    Starting from base_pose, iterate over the list of parameters in 'params'
    (e.g., ['yaw','pitch','roll','x','y','z']). For each parameter, vary that
    parameter (while keeping others fixed) until the FoV is lost. The last valid
    pose for that parameter is recorded as the boundary. The current pose is then
    updated to that boundary before moving on to the next parameter.
    
    Returns:
      - trajectory: a list of 6D poses (base_pose and subsequent boundary poses)
      - boundaries: a dictionary with the allowed variation for each parameter.
    """
    trajectory = [base_pose.copy()]
    boundaries = {}
    current_pose = base_pose.copy()
    for param in params:
        neg_valid, pos_valid = get_allowed_variation(current_pose, param, step_dict[param], max_range_dict[param], goal, camera_fov)
        boundaries[param] = (neg_valid, pos_valid)
        # For synthesis, choose the midpoint (typically zero, but if already varied, use pos_valid)
        # Here we simply update current_pose by moving in the positive direction up to the boundary.
        current_pose[param_to_index[param]] += pos_valid
        trajectory.append(current_pose.copy())
    return trajectory, boundaries

###############################################################################
# 5) Main: Load Data, Synthesize Trajectory, and Plot Boundary
###############################################################################

def main():
    # Load main trajectory data from file.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Use the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Use the first point of the main trajectory as our base pose.
    base_pos = positions_list[0][0]       # [x, y, z]
    base_rpy = orientations_list[0][0]      # [roll, pitch, yaw]
    base_pose = list(base_pos) + list(base_rpy)  # 6D pose
    
    # Define the goal as the last point of the main trajectory.
    goal = positions_list[0][-1]  # [x, y, z]
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # Define which parameters to vary (order matters).
    # For example, vary yaw first, then pitch, then roll, then x, y, z.
    params = ['yaw', 'pitch', 'roll', 'x', 'y', 'z']
    
    # Define step sizes for each parameter.
    deg_to_rad = np.pi / 180.0
    step_dict = {
        'x': 0.001, 'y': 0.001, 'z': 0.001,
        'roll': 0.5 * deg_to_rad,
        'pitch': 0.5 * deg_to_rad,
        'yaw': 0.5 * deg_to_rad
    }
    # Define maximum variation ranges.
    max_range_dict = {
        'x': 0.05, 'y': 0.05, 'z': 0.05,
        'roll': 10 * deg_to_rad,
        'pitch': 10 * deg_to_rad,
        'yaw': 10 * deg_to_rad
    }
    
    # Synthesize a new trajectory by varying each parameter.
    trajectory, boundaries = synthesize_trajectory(base_pose, goal, camera_fov, params, step_dict, max_range_dict)
    
    print("Synthesized Trajectory (6D poses):")
    for pose in trajectory:
        print(pose)
    
    print("\nAllowed Variation Boundaries:")
    for p in boundaries:
        print(f"{p}: {boundaries[p]}")
    
    # Now, to visualize the boundary in 3D space, we can generate a grid of candidate poses
    # using the allowed ranges for each parameter. Here, we vary each parameter in a coarse grid.
    grid_poses = []
    num_steps = 5  # grid resolution per parameter
    # For simplicity, we vary only the angular parameters here.
    for yaw_delta in np.linspace(boundaries['yaw'][0], boundaries['yaw'][1], num_steps):
        for pitch_delta in np.linspace(boundaries['pitch'][0], boundaries['pitch'][1], num_steps):
            for roll_delta in np.linspace(boundaries['roll'][0], boundaries['roll'][1], num_steps):
                # Keep position fixed as in the synthesized trajectory's last position.
                pos = trajectory[-1][:3]
                candidate_pose = [pos[0], pos[1], pos[2],
                                  trajectory[-1][3] + yaw_delta,  # note: our order is [roll, pitch, yaw]
                                  trajectory[-1][4] + pitch_delta,
                                  trajectory[-1][5] + roll_delta]
                cam_pose = candidate_to_cam_pose(candidate_pose)
                if is_inFOV(cam_pose, camera_fov, goal):
                    grid_poses.append(candidate_pose)
    
    # Plot the synthesized trajectory and the valid boundary candidates.
    traj_x = [p[0] for p in trajectory]
    traj_y = [p[1] for p in trajectory]
    traj_z = [p[2] for p in trajectory]
    
    grid_x = [p[0] for p in grid_poses]
    grid_y = [p[1] for p in grid_poses]
    grid_z = [p[2] for p in grid_poses]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=traj_x, y=traj_y, z=traj_z,
        mode='lines+markers',
        marker=dict(size=6, color='green'),
        name='Synthesized Trajectory'
    ))
    fig.add_trace(go.Scatter3d(
        x=grid_x, y=grid_y, z=grid_z,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Valid Variation Boundary'
    ))
    # Mark the goal position.
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Synthesized Trajectory and Variation Boundaries",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("synthesized_trajectory_boundary.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Use Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given the cube center (cx,cy,cz) and a rotation (computed with the 'zyx' convention),
    compute the 8 corner points of a cube.
    
    Returns:
      An (8 x 3) numpy array of the cube corner positions in world coordinates.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) FoV Checker and Image Generation Helper
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    """
    Generate a synthetic camera image: a black image with a white dot indicating
    the projected location of the goal.
    
    Assumes:
      - camera_pose: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x-axis.
      - Euler angles are interpreted with the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    # Transform the goal into the camera frame.
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    # Compute angles relative to the x-axis.
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    # Map these angles into image pixel coordinates.
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose is a 7-tuple: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x-axis.
      - RPY values are interpreted with the 'zyx' (yaw-pitch-roll) convention.
    
    Returns True if the goal is in FoV, else False.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Parameter Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x, y, z, roll, pitch, yaw] into a 7-tuple
    camera pose (x, y, z, qx, qy, qz, qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

param_to_index = {'x': 0, 'y': 1, 'z': 2, 'roll': 3, 'pitch': 4, 'yaw': 5}

def get_allowed_variation(base_pose, param, step, max_range, goal, camera_fov):
    """
    Starting from base_pose (a list of 6 values: [x,y,z,roll,pitch,yaw]),
    vary the specified parameter (one of 'x', 'y', 'z', 'roll', 'pitch', 'yaw')
    in both the positive and negative directions (using increments of step) until
    the FoV check fails. Returns (min_delta, max_delta) where these deltas are relative
    to base_pose.
    """
    idx = param_to_index[param]
    
    # Test positive direction.
    delta = 0.0
    pos_valid = 0.0
    while abs(delta) <= max_range:
        candidate = base_pose.copy()
        candidate[idx] += delta
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            pos_valid = delta
            delta += step
        else:
            break
    
    # Test negative direction.
    delta = 0.0
    neg_valid = 0.0
    while abs(delta) <= max_range:
        candidate = base_pose.copy()
        candidate[idx] -= delta
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            neg_valid = -delta
            delta += step
        else:
            break
    return neg_valid, pos_valid

def synthesize_pose_for_point(base_pose, goal, camera_fov, params, step_dict, max_range_dict):
    """
    Starting from base_pose (a list of 6 values), iterate over each parameter in the
    order given by 'params'. For each parameter, vary that parameter until the FoV check
    fails (in the positive direction) and update the base_pose to the maximum valid variation.
    
    Returns:
      - synthesized_pose: the updated 6D pose (at the boundary for each parameter)
      - boundaries: a dictionary mapping each parameter to its allowed variation (neg, pos)
    """
    synthesized_pose = base_pose.copy()
    boundaries = {}
    for param in params:
        neg_valid, pos_valid = get_allowed_variation(synthesized_pose, param, step_dict[param], max_range_dict[param], goal, camera_fov)
        boundaries[param] = (neg_valid, pos_valid)
        # For synthesis, update the pose by moving in the positive direction up to pos_valid.
        synthesized_pose[param_to_index[param]] += pos_valid
    return synthesized_pose, boundaries

###############################################################################
# 5) Main: Process All Main Trajectory Points and Synthesize New Trajectory
###############################################################################

def main():
    # Load main trajectory data from file.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Use the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # For each point in the main trajectory, form a 6D pose [x,y,z,roll,pitch,yaw].
    main_traj = []
    for pos, rpy in zip(positions_list[0], orientations_list[0]):
        main_traj.append(list(pos) + list(rpy))
    
    # Define the goal as the last point of the main trajectory.
    goal = positions_list[0][-1]  # [x, y, z]
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # Define the order in which to vary parameters.
    params = ['yaw', 'pitch', 'roll', 'x', 'y', 'z']
    
    # Define step sizes and maximum variation ranges.
    deg_to_rad = np.pi / 180.0
    step_dict = {
        'x': 0.001, 'y': 0.001, 'z': 0.001,
        'roll': 0.5 * deg_to_rad,
        'pitch': 0.5 * deg_to_rad,
        'yaw': 0.5 * deg_to_rad
    }
    max_range_dict = {
        'x': 0.05, 'y': 0.05, 'z': 0.05,
        'roll': 10 * deg_to_rad,
        'pitch': 10 * deg_to_rad,
        'yaw': 10 * deg_to_rad
    }
    
    # Synthesize a new trajectory by processing each point of the main trajectory.
    synthesized_traj = []
    boundaries_all = []
    for base_pose in main_traj:
        synth_pose, boundaries = synthesize_pose_for_point(base_pose, goal, camera_fov, params, step_dict, max_range_dict)
        synthesized_traj.append(synth_pose)
        boundaries_all.append(boundaries)
    
    # Print synthesized trajectory and boundaries.
    print("Synthesized Trajectory (6D poses):")
    for pose in synthesized_traj:
        print(pose)
    print("\nAllowed Variation Boundaries (per main trajectory point):")
    for i, b in enumerate(boundaries_all):
        print(f"Point {i}: {b}")
    
    # Plot the synthesized trajectory in 3D.
    traj_x = [pose[0] for pose in synthesized_traj]
    traj_y = [pose[1] for pose in synthesized_traj]
    traj_z = [pose[2] for pose in synthesized_traj]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=traj_x, y=traj_y, z=traj_z,
        mode='lines+markers',
        marker=dict(size=6, color='green'),
        name='Synthesized Trajectory'
    ))
    # Mark the goal position.
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Goal'
    ))
    fig.update_layout(
        title="Synthesized Trajectory from Main Trajectory Points",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("synthesized_trajectory_all_points.html")
    fig.show()
    
    # As an example, display a grid (boundary envelope) for the first trajectory point.
    base = main_traj[0]
    synth, boundaries = synthesize_pose_for_point(base, goal, camera_fov, params, step_dict, max_range_dict)
    # For the angular parameters, generate a coarse grid over the allowed range.
    grid_poses = []
    num_steps = 5
    for yaw_delta in np.linspace(boundaries['yaw'][0], boundaries['yaw'][1], num_steps):
        for pitch_delta in np.linspace(boundaries['pitch'][0], boundaries['pitch'][1], num_steps):
            for roll_delta in np.linspace(boundaries['roll'][0], boundaries['roll'][1], num_steps):
                candidate = base.copy()
                candidate[param_to_index['yaw']] += yaw_delta
                candidate[param_to_index['pitch']] += pitch_delta
                candidate[param_to_index['roll']] += roll_delta
                cam_pose = candidate_to_cam_pose(candidate)
                if is_inFOV(cam_pose, camera_fov, goal):
                    grid_poses.append(candidate)
    
    grid_x = [p[0] for p in grid_poses]
    grid_y = [p[1] for p in grid_poses]
    grid_z = [p[2] for p in grid_poses]
    
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter3d(
        x=grid_x, y=grid_y, z=grid_z,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Valid Variation Boundary (for 1st Point)'
    ))
    fig2.add_trace(go.Scatter3d(
        x=[base[0]], y=[base[1]], z=[base[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Base Pose'
    ))
    fig2.update_layout(
        title="Variation Boundary for First Trajectory Point",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig2.write_html("boundary_first_point.html")
    fig2.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given the cube center (cx,cy,cz) and a rotation (computed with the 'zyx' convention),
    compute the 8 corner points of a cube.
    
    Returns:
      An (8 x 3) numpy array of world coordinates for the cube's corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) FoV Checker and Image Generation Helper
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    """
    Generate a synthetic camera image: a black image with a white dot where the goal projects.
    Assumes:
      - camera_pose: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x-axis.
      - RPY are interpreted with the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    Assumptions:
      - camera_pose: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x-axis.
      - RPY values are interpreted using the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x, y, z, roll, pitch, yaw] into a 7-tuple camera pose
    (x, y, z, qx, qy, qz, qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

param_to_index = {'x': 0, 'y': 1, 'z': 2, 'roll': 3, 'pitch': 4, 'yaw': 5}

def get_allowed_variation(base_pose, param, step, max_range, goal, camera_fov):
    """
    Starting from base_pose (a list of 6 values: [x,y,z,roll,pitch,yaw]),
    vary the specified parameter (e.g., 'x', 'roll', etc.) in small steps until the
    FoV check fails. Returns (min_delta, max_delta) for that parameter.
    """
    idx = param_to_index[param]
    # Test positive direction.
    delta = 0.0
    pos_valid = 0.0
    while abs(delta) <= max_range:
        candidate = base_pose.copy()
        candidate[idx] += delta
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            pos_valid = delta
            delta += step
        else:
            break
    # Test negative direction.
    delta = 0.0
    neg_valid = 0.0
    while abs(delta) <= max_range:
        candidate = base_pose.copy()
        candidate[idx] -= delta
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            neg_valid = -delta
            delta += step
        else:
            break
    return neg_valid, pos_valid

def synthesize_pose_for_point(base_pose, goal, camera_fov, params, step_dict, max_range_dict):
    """
    Starting from a base_pose (6D: [x,y,z,roll,pitch,yaw]) of a main trajectory point,
    vary each parameter (in the order given by 'params') until the FoV check fails.
    Update the base_pose to the maximum valid value (in the positive direction) for each parameter.
    
    Returns:
      - synthesized_pose: the updated 6D pose for this trajectory point.
      - boundaries: a dictionary mapping each parameter to its allowed (min, max) variation.
    """
    synthesized_pose = base_pose.copy()
    boundaries = {}
    for param in params:
        neg_valid, pos_valid = get_allowed_variation(synthesized_pose, param, step_dict[param], max_range_dict[param], goal, camera_fov)
        boundaries[param] = (neg_valid, pos_valid)
        # Update the synthesized pose by pushing in the positive direction.
        synthesized_pose[param_to_index[param]] += pos_valid
    return synthesized_pose, boundaries

###############################################################################
# 5) Main: Process Multiple Main Trajectories and Synthesize New Trajectories
###############################################################################

def main():
    # Load all main trajectories from file.
    file_path = "demo_duck_feb12.hdf5"
    main_trajs = []  # each element will be a list of 6D poses.
    goals = []       # corresponding goal (last point) for each trajectory.
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Process all demos.
        for demo_name in demo_keys:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T, 7)
                traj = []
                for pos, rpy in zip(robot_states[:, :3], robot_states[:, 3:6]):
                    traj.append(list(pos) + list(rpy))
                main_trajs.append(traj)
                goals.append(robot_states[-1, :3])
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # Parameter variation order.
    params = ['yaw', 'pitch', 'roll', 'x', 'y', 'z']
    
    # Step sizes and maximum variation ranges.
    deg_to_rad = np.pi / 180.0
    step_dict = {
        'x': 0.001, 'y': 0.001, 'z': 0.001,
        'roll': 0.5 * deg_to_rad,
        'pitch': 0.5 * deg_to_rad,
        'yaw': 0.5 * deg_to_rad
    }
    max_range_dict = {
        'x': 0.05, 'y': 0.05, 'z': 0.05,
        'roll': 10 * deg_to_rad,
        'pitch': 10 * deg_to_rad,
        'yaw': 10 * deg_to_rad
    }
    
    synthesized_trajs = []   # list of synthesized trajectories (one per main trajectory)
    boundaries_all = []      # corresponding boundaries for each trajectory
    for traj, goal in zip(main_trajs, goals):
        synth_traj = []
        traj_boundaries = []
        for base_pose in traj:
            synth_pose, boundaries = synthesize_pose_for_point(base_pose, goal, camera_fov, params, step_dict, max_range_dict)
            synth_traj.append(synth_pose)
            traj_boundaries.append(boundaries)
        synthesized_trajs.append(synth_traj)
        boundaries_all.append(traj_boundaries)
    
    # Print synthesized trajectories.
    for t_idx, synth_traj in enumerate(synthesized_trajs):
        print(f"\nSynthesized Trajectory {t_idx}:")
        for pose in synth_traj:
            print(pose)
    
    # Plot all synthesized trajectories in 3D with different colors.
    colors = ['green', 'orange', 'purple', 'cyan', 'magenta']
    fig = go.Figure()
    for i, synth_traj in enumerate(synthesized_trajs):
        traj_x = [p[0] for p in synth_traj]
        traj_y = [p[1] for p in synth_traj]
        traj_z = [p[2] for p in synth_traj]
        fig.add_trace(go.Scatter3d(
            x=traj_x, y=traj_y, z=traj_z,
            mode='lines+markers',
            marker=dict(size=4, color=colors[i % len(colors)]),
            name=f'Synthesized Traj {i}'
        ))
        # Also mark the goal for each trajectory.
        goal = goals[i]
        fig.add_trace(go.Scatter3d(
            x=[goal[0]], y=[goal[1]], z=[goal[2]],
            mode='markers',
            marker=dict(size=8, color='blue'),
            name=f'Goal {i}'
        ))
    
    fig.update_layout(
        title="Synthesized Trajectories from Main Trajectory Points",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("synthesized_trajectories_multiple.html")
    fig.show()
    
    # For illustration, for the first main trajectory, generate a grid over allowed angular variations
    # for its first point.
    base = main_trajs[0][0]
    synth, boundaries = synthesize_pose_for_point(base, goals[0], camera_fov, params, step_dict, max_range_dict)
    grid_poses = []
    num_steps = 5  # grid resolution for angular parameters
    for yaw_delta in np.linspace(boundaries['yaw'][0], boundaries['yaw'][1], num_steps):
        for pitch_delta in np.linspace(boundaries['pitch'][0], boundaries['pitch'][1], num_steps):
            for roll_delta in np.linspace(boundaries['roll'][0], boundaries['roll'][1], num_steps):
                candidate = base.copy()
                candidate[param_to_index['yaw']] += yaw_delta
                candidate[param_to_index['pitch']] += pitch_delta
                candidate[param_to_index['roll']] += roll_delta
                cam_pose = candidate_to_cam_pose(candidate)
                if is_inFOV(cam_pose, camera_fov, goals[0]):
                    grid_poses.append(candidate)
    grid_x = [p[0] for p in grid_poses]
    grid_y = [p[1] for p in grid_poses]
    grid_z = [p[2] for p in grid_poses]
    
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter3d(
        x=grid_x, y=grid_y, z=grid_z,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Valid Variation Boundary (1st Point)'
    ))
    fig2.add_trace(go.Scatter3d(
        x=[base[0]], y=[base[1]], z=[base[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Base Pose (1st Point)'
    ))
    fig2.update_layout(
        title="Variation Boundary for First Trajectory Point",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig2.write_html("boundary_first_point_multiple.html")
    fig2.show()
    
    # Optionally, you could store the synthesized trajectories and boundaries in a file or dataset.

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given cube center (cx,cy,cz) and a rotation (from R.from_euler('zyx', ...)),
    compute the eight corner points of the cube.
    Returns an (8 x 3) numpy array of candidate viewpoint positions.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker and Helper for Camera Image (not used for trajectory)
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj)-cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj)-cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x, y, z, roll, pitch, yaw] into a 7-tuple camera pose:
    (x, y, z, qx, qy, qz, qw) using the 'zyx' conversion.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

param_to_index = {'x': 0, 'y': 1, 'z': 2, 'roll': 3, 'pitch': 4, 'yaw': 5}

###############################################################################
# 5) Candidate Generation for Each Main Trajectory Point
###############################################################################

def generate_candidates_for_point(base_pose, goal, camera_fov, 
                                  step_dict, max_range_dict, num_samples=10):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses 
    by adding random variations in spherical coordinates (for position) and Euler angles.
    
    Returns:
      A list of candidate poses (each a 6D list) that have the FoV flag True.
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    # Use an exponential decay factor based on the index if desired;
    # here we assume uniform variation per point.
    fraction = 1.0  # You could adjust this if needed.
    
    for _ in range(num_samples):
        # Unpack base_pose: [x,y,z,roll,pitch,yaw]
        x, y, z, roll, pitch, yaw = base_pose
        # Position variation: compute difference to goal in spherical coordinates.
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        delta_angle = fraction * (max_range_dict['x'] * deg_to_rad)  # not used directly
        # For simplicity, add a random offset within ±(max_deg) for the spherical angles.
        # (Here, max_deg can be set in step_dict if needed; we'll use max_range_dict for now)
        az_var = az + random.uniform(-max_range_dict['roll'], max_range_dict['roll'])*deg_to_rad
        el_var = el + random.uniform(-max_range_dict['pitch'], max_range_dict['pitch'])*deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        # Orientation variation: add random offset for roll, pitch, yaw within ±max_range.
        droll = random.uniform(-max_range_dict['roll'], max_range_dict['roll'])
        dpitch = random.uniform(-max_range_dict['pitch'], max_range_dict['pitch'])
        dyaw = random.uniform(-max_range_dict['yaw'], max_range_dict['yaw'])
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

###############################################################################
# 6) Main: Generate Synthesized Trajectory by Randomly Selecting from Candidates
###############################################################################

def main():
    # Load main trajectory data from file.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # For simplicity, process the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = demo_data["obs"]["joint_states"][:]  # shape: (T,7)
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Form main trajectory as list of 6D poses.
    main_traj = []
    for pos, rpy in zip(positions_list[0], orientations_list[0]):
        main_traj.append(list(pos) + list(rpy))
    
    # Define the goal as the last point's position.
    goal = positions_list[0][-1]
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # Variation parameters (step sizes and maximum range for randomness).
    # Here we use the following maximum ranges (in our candidate generation, we use these as random offsets)
    deg_to_rad = np.pi / 180.0
    step_dict = {
        'x': 0.001, 'y': 0.001, 'z': 0.001,
        'roll': 0.5 * deg_to_rad,
        'pitch': 0.5 * deg_to_rad,
        'yaw': 0.5 * deg_to_rad
    }
    max_range_dict = {
        'x': 0.05, 'y': 0.05, 'z': 0.05,
        'roll': 10,    # in degrees
        'pitch': 10,   # in degrees
        'yaw': 10      # in degrees
    }
    
    # For each main trajectory point, generate candidate poses.
    candidates_by_point = {}
    for i, base_pose in enumerate(main_traj):
        cands = generate_candidates_for_point(base_pose, goal, camera_fov, step_dict, max_range_dict, num_samples=10)
        candidates_by_point[i] = cands
    
    # Now, for synthesizing a new trajectory, choose one candidate (randomly) from each point's candidate set.
    synthesized_traj = []
    for i in range(len(main_traj)):
        cands = candidates_by_point.get(i, [])
        if cands:
            chosen = random.choice(cands)
            synthesized_traj.append(chosen)
        else:
            # Fallback: use the original main trajectory pose.
            synthesized_traj.append(main_traj[i])
    # Ensure the trajectory ends exactly at the goal.
    synthesized_traj[-1][:3] = list(goal)
    
    # Plot main trajectory and synthesized trajectory.
    main_x = [pose[0] for pose in main_traj]
    main_y = [pose[1] for pose in main_traj]
    main_z = [pose[2] for pose in main_traj]
    
    synth_x = [pose[0] for pose in synthesized_traj]
    synth_y = [pose[1] for pose in synthesized_traj]
    synth_z = [pose[2] for pose in synthesized_traj]
    
    fig = go.Figure()
    # Plot main trajectory with a specific color (e.g., red)
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        marker=dict(size=6, color='red'),
        name='Main Trajectory'
    ))
    # Plot synthesized trajectory in green.
    fig.add_trace(go.Scatter3d(
        x=synth_x, y=synth_y, z=synth_z,
        mode='lines+markers',
        marker=dict(size=6, color='green'),
        name='Synthesized Trajectory'
    ))
    # Mark the goal position.
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=10, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Main Trajectory (Red) and Synthesized Trajectory (Green)",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("synthesized_trajectory_final.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    """
    Convert Cartesian (x, y, z) -> Spherical (r, azimuth, elevation)
    """
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    """
    Convert Spherical (r, azimuth, elevation) -> Cartesian (x, y, z)
    """
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given cube center (cx,cy,cz) and a rotation (using the 'zyx' convention),
    compute the eight corner points of the cube.
    
    Returns:
      An (8 x 3) array of world coordinates for the cube's corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker (and camera image helper – not used in synthesis)
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    """
    Generate a synthetic image (black with white dot) showing where the goal projects.
    (This helper is included if you wish to store images; it is not used in trajectory synthesis.)
    """
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x‑axis.
      - RPY values are interpreted using the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x, y, z, roll, pitch, yaw] into a 7-tuple camera pose
    (x, y, z, qx, qy, qz, qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

# Mapping parameter names to indices in our 6D pose.
param_to_index = {'x': 0, 'y': 1, 'z': 2, 'roll': 3, 'pitch': 4, 'yaw': 5}

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses
    (using the cube-corner method) with random variations.
    
    Returns a list of candidate 6D poses (each is [x,y,z,roll,pitch,yaw]) that pass the FoV check.
    """
    candidates = []
    # We apply random variations for both position and orientation.
    deg_to_rad = np.pi / 180.0
    # Use maximum variation ranges (in degrees for angles, in meters for position).
    max_pos_range = 0.05
    max_ang_range = 10  # in degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        # Randomly perturb position (in spherical coordinates relative to goal).
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        # Add a random offset within ±max_ang_range (converted to radians) for az and el.
        az_var = az + random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        # Randomly perturb orientation.
        droll = random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        dyaw = random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

def get_forward_vector(candidate):
    """
    Given a 6D candidate pose [x, y, z, roll, pitch, yaw], return the camera's
    forward vector (optical axis) in world coordinates.
    """
    cam_pose = candidate_to_cam_pose(candidate)
    quat = cam_pose[3:]
    rot = R.from_quat(quat).as_matrix()
    # Optical axis is the positive x-axis in the camera frame.
    forward = rot @ np.array([1, 0, 0])
    return forward

def select_best_candidate(prev_candidate, candidate_list):
    """
    Given the previously selected candidate (prev_candidate) and a list of candidate
    poses for the next trajectory point, choose the candidate that is most aligned.
    
    Alignment is computed as the dot product between:
      - The forward vector of prev_candidate.
      - The normalized displacement vector from prev_candidate's position to the candidate's position.
    """
    if not candidate_list:
        return None
    best_candidate = None
    best_score = -np.inf
    p_pos = np.array(prev_candidate[:3])
    p_forward = get_forward_vector(prev_candidate)
    for cand in candidate_list:
        cand_pos = np.array(cand[:3])
        d = cand_pos - p_pos
        norm_d = np.linalg.norm(d)
        if norm_d < 1e-6:
            continue
        d_normalized = d / norm_d
        score = np.dot(d_normalized, p_forward)
        if score > best_score:
            best_score = score
            best_candidate = cand
    return best_candidate

###############################################################################
# 6) Main: Synthesize Trajectory by Chaining Best-Aligned Candidates
###############################################################################

def main():
    # Load main trajectories from file.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Process the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T,7)
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Form the main trajectory as a list of 6D poses.
    main_traj = []
    for pos, rpy in zip(positions_list[0], orientations_list[0]):
        main_traj.append(list(pos) + list(rpy))
    
    # Define the goal as the last point's position.
    goal = positions_list[0][-1]
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # Generate candidate poses for each main trajectory point.
    candidates_by_point = {}
    for i, base_pose in enumerate(main_traj):
        cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10)
        candidates_by_point[i] = cands
    
    # Synthesize a trajectory by chaining candidates.
    synthesized_traj = []
    # For the first point, choose the candidate that is closest to the main pose.
    if candidates_by_point.get(0):
        # Here we simply choose the candidate with minimum Euclidean distance to the main pose.
        base = np.array(main_traj[0])
        best = min(candidates_by_point[0], key=lambda c: np.linalg.norm(np.array(c[:3]) - base[:3]))
        synthesized_traj.append(best)
    else:
        synthesized_traj.append(main_traj[0])
    
    # For subsequent points, choose the candidate that is most aligned with the previous candidate.
    for i in range(1, len(main_traj)):
        cand_list = candidates_by_point.get(i, [])
        best = select_best_candidate(synthesized_traj[-1], cand_list)
        if best is None:
            # If no candidate is available, fall back to main trajectory pose.
            synthesized_traj.append(main_traj[i])
        else:
            synthesized_traj.append(best)
    
    # Force the last point to be the goal.
    synthesized_traj[-1][:3] = list(goal)
    
    # Plot main trajectory (red) and synthesized trajectory (green).
    main_x = [pose[0] for pose in main_traj]
    main_y = [pose[1] for pose in main_traj]
    main_z = [pose[2] for pose in main_traj]
    
    synth_x = [pose[0] for pose in synthesized_traj]
    synth_y = [pose[1] for pose in synthesized_traj]
    synth_z = [pose[2] for pose in synthesized_traj]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        marker=dict(size=6, color='red'),
        name='Main Trajectory'
    ))
    fig.add_trace(go.Scatter3d(
        x=synth_x, y=synth_y, z=synth_z,
        mode='lines+markers',
        marker=dict(size=6, color='green'),
        name='Synthesized Trajectory'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=10, color='blue'),
        name='Goal'
    ))
    
    fig.update_layout(
        title="Main Trajectory (Red) and Synthesized Trajectory (Green)",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("synthesized_trajectory_aligned.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given the cube center (cx,cy,cz) and a rotation (computed with 'zyx'),
    compute the eight corner points of the cube.
    
    Returns:
      An (8 x 3) numpy array of candidate viewpoint positions.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker and Camera Image Generator (for reference)
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj)-cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj)-cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x,y,z,roll,pitch,yaw] into a 7-tuple camera pose
    (x,y,z,qx,qy,qz,qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

# Parameter name to index mapping.
param_to_index = {'x':0, 'y':1, 'z':2, 'roll':3, 'pitch':4, 'yaw':5}

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses
    using the cube-corner method with random variations. Only returns candidates that pass
    the FoV check.
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    # Define maximum random variations.
    max_pos_range = 0.05
    max_ang_range = 10  # degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        # Perturb position relative to the goal.
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        az_var = az + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        # Perturb orientation.
        droll = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dyaw = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

def get_forward_vector(candidate):
    """
    Given a candidate 6D pose, return its camera forward (optical) vector in world coordinates.
    """
    cam_pose = candidate_to_cam_pose(candidate)
    quat = cam_pose[3:]
    rot = R.from_quat(quat).as_matrix()
    # Optical axis is along positive x-axis.
    forward = rot @ np.array([1, 0, 0])
    return forward

def select_best_candidate(prev_candidate, candidate_list):
    """
    Given the previously selected candidate and a list of candidates for the next trajectory point,
    choose the candidate that minimizes a composite cost.
    
    The cost is defined as:
       cost = (Euclidean distance between positions) +
              (w1 * angular difference between forward vectors) +
              (w2 * angular difference between the displacement vector (from prev to candidate)
                     and the previous candidate's forward vector)
    
    Lower cost indicates better alignment.
    """
    if not candidate_list:
        return None
    best_candidate = None
    best_score = float('inf')
    p_pos = np.array(prev_candidate[:3])
    p_forward = get_forward_vector(prev_candidate)
    for cand in candidate_list:
        cand_pos = np.array(cand[:3])
        pos_diff = np.linalg.norm(cand_pos - p_pos)
        # Candidate's forward vector.
        cand_forward = get_forward_vector(cand)
        # Angular difference between forward vectors.
        dot_f = np.clip(np.dot(p_forward, cand_forward), -1.0, 1.0)
        ang_diff = np.arccos(dot_f)
        # Displacement vector from prev_candidate to candidate.
        d = cand_pos - p_pos
        if np.linalg.norm(d) < 1e-6:
            disp_ang = 0.0
        else:
            d_norm = d / np.linalg.norm(d)
            dot_disp = np.clip(np.dot(p_forward, d_norm), -1.0, 1.0)
            disp_ang = np.arccos(dot_disp)
        # Weigh the components; these weights can be tuned.
        w1 = 0.1
        w2 = 0.1
        cost = pos_diff + w1 * ang_diff + w2 * disp_ang
        if cost < best_score:
            best_score = cost
            best_candidate = cand
    return best_candidate

###############################################################################
# 7) Main: Synthesize Trajectory by Chaining Best-Aligned Candidates
###############################################################################

def main():
    # Load main trajectory data from file.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Process the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = demo_data["obs"]["joint_states"][:]  # shape: (T,7)
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Form main trajectory as a list of 6D poses.
    main_traj = []
    for pos, rpy in zip(positions_list[0], orientations_list[0]):
        main_traj.append(list(pos) + list(rpy))
    
    # Define the goal as the last point of the main trajectory.
    goal = positions_list[0][-1]
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # For each main trajectory point, generate candidate poses.
    candidates_by_point = {}
    for i, base_pose in enumerate(main_traj):
        cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10)
        candidates_by_point[i] = cands
    
    # Synthesize a trajectory by chaining candidates.
    synthesized_traj = []
    # For the first main trajectory point, choose the candidate closest in position to the main pose.
    if candidates_by_point.get(0):
        base = np.array(main_traj[0])
        best = min(candidates_by_point[0], key=lambda c: np.linalg.norm(np.array(c[:3]) - base[:3]))
        synthesized_traj.append(best)
    else:
        synthesized_traj.append(main_traj[0])
    
    # For each subsequent point, choose the candidate that is most aligned with the previous selected candidate.
    for i in range(1, len(main_traj)):
        cand_list = candidates_by_point.get(i, [])
        best = select_best_candidate(synthesized_traj[-1], cand_list)
        if best is None:
            synthesized_traj.append(main_traj[i])
        else:
            synthesized_traj.append(best)
    
    # Force the last point to be the goal.
    synthesized_traj[-1][:3] = list(goal)
    
    # Plot main trajectory (red) and synthesized trajectory (green).
    main_x = [pose[0] for pose in main_traj]
    main_y = [pose[1] for pose in main_traj]
    main_z = [pose[2] for pose in main_traj]
    
    synth_x = [pose[0] for pose in synthesized_traj]
    synth_y = [pose[1] for pose in synthesized_traj]
    synth_z = [pose[2] for pose in synthesized_traj]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        marker=dict(size=6, color='red'),
        name='Main Trajectory'
    ))
    fig.add_trace(go.Scatter3d(
        x=synth_x, y=synth_y, z=synth_z,
        mode='lines+markers',
        marker=dict(size=6, color='green'),
        name='Synthesized Trajectory'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=10, color='blue'),
        name='Goal'
    ))
    fig.update_layout(
        title="Main Trajectory (Red) and Synthesized Trajectory (Green)",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("synthesized_trajectory_aligned_smooth.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given cube center (cx,cy,cz) and a rotation (using the 'zyx' convention),
    compute the eight corner points of the cube.
    
    Returns an (8 x 3) array of world coordinates for the cube's corners.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker (and helper to generate a camera image if needed)
###############################################################################

def generate_camera_image(camera_pose, camera_fov, point_obj, img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, (img_x, img_y), dot_dia//2, (255,255,255), -1)
    return image

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Variation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x,y,z,roll,pitch,yaw] into a 7-tuple camera pose
    (x,y,z,qx,qy,qz,qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

param_to_index = {'x':0, 'y':1, 'z':2, 'roll':3, 'pitch':4, 'yaw':5}

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses
    using the cube-corner method with random variations. Only returns candidates that
    pass the FoV check.
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    max_pos_range = 0.05
    max_ang_range = 10  # degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        az_var = az + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        droll = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dyaw = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

def get_forward_vector(candidate):
    """
    Given a candidate 6D pose, return its camera forward vector (optical axis)
    in world coordinates.
    """
    cam_pose = candidate_to_cam_pose(candidate)
    quat = cam_pose[3:]
    rot = R.from_quat(quat).as_matrix()
    forward = rot @ np.array([1, 0, 0])
    return forward

def select_best_candidate(prev_candidate, candidate_list):
    """
    Given the previously selected candidate and a list of candidates for the next main trajectory point,
    choose the candidate that minimizes a composite cost. The cost here is defined as the sum of:
       - The Euclidean distance between the candidate's position and the previous candidate's position.
       - A weighted angular difference between the candidate's forward vector and the previous candidate's forward vector.
    
    This cost helps select a candidate that is smoothly aligned with the previous candidate.
    """
    if not candidate_list:
        return None
    best_candidate = None
    best_score = float('inf')
    p_pos = np.array(prev_candidate[:3])
    p_forward = get_forward_vector(prev_candidate)
    for cand in candidate_list:
        cand_pos = np.array(cand[:3])
        pos_diff = np.linalg.norm(cand_pos - p_pos)
        cand_forward = get_forward_vector(cand)
        dot_val = np.clip(np.dot(p_forward, cand_forward), -1.0, 1.0)
        ang_diff = np.arccos(dot_val)
        # Here we use weights to balance position and orientation differences.
        cost = pos_diff + 0.1 * ang_diff
        if cost < best_score:
            best_score = cost
            best_candidate = cand
    return best_candidate

###############################################################################
# 5) Main: Synthesize Trajectories (One per Main Trajectory) Separately
###############################################################################

def main():
    # Load main trajectories from file.
    file_path = "demo_duck_feb12.hdf5"
    main_trajs = []  # list of main trajectories, each a list of 6D poses.
    goals = []       # goal (last point position) for each trajectory.
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T,7)
                traj = []
                for pos, rpy in zip(robot_states[:, :3], robot_states[:, 3:6]):
                    traj.append(list(pos) + list(rpy))
                main_trajs.append(traj)
                goals.append(robot_states[-1, :3])
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # For each main trajectory, generate candidates at each point.
    synthesized_trajs = []  # List of synthesized trajectories (one per main trajectory).
    for traj, goal in zip(main_trajs, goals):
        candidates_by_point = {}
        for i, base_pose in enumerate(traj):
            cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10)
            candidates_by_point[i] = cands
        
        # Synthesize one trajectory by chaining candidates that are best aligned.
        synth_traj = []
        # For the first point, pick the candidate closest to the main pose.
        if candidates_by_point.get(0):
            base = np.array(traj[0])
            best = min(candidates_by_point[0], key=lambda c: np.linalg.norm(np.array(c[:3]) - base[:3]))
            synth_traj.append(best)
        else:
            synth_traj.append(traj[0])
        # For subsequent points, select candidate that minimizes the composite cost.
        for i in range(1, len(traj)):
            cand_list = candidates_by_point.get(i, [])
            best = select_best_candidate(synth_traj[-1], cand_list)
            if best is None:
                synth_traj.append(traj[i])
            else:
                synth_traj.append(best)
        # Force last point to be the goal.
        synth_traj[-1][:3] = list(goal)
        synthesized_trajs.append(synth_traj)
    
    # Now, plot each trajectory separately.
    for t_idx, (traj, synth_traj, goal) in enumerate(zip(main_trajs, synthesized_trajs, goals)):
        main_x = [pose[0] for pose in traj]
        main_y = [pose[1] for pose in traj]
        main_z = [pose[2] for pose in traj]
        synth_x = [pose[0] for pose in synth_traj]
        synth_y = [pose[1] for pose in synth_traj]
        synth_z = [pose[2] for pose in synth_traj]
        
        fig = go.Figure()
        # Plot main trajectory (red).
        fig.add_trace(go.Scatter3d(
            x=main_x, y=main_y, z=main_z,
            mode='lines+markers',
            marker=dict(size=6, color='red'),
            name='Main Trajectory'
        ))
        # Plot synthesized trajectory (green).
        fig.add_trace(go.Scatter3d(
            x=synth_x, y=synth_y, z=synth_z,
            mode='lines+markers',
            marker=dict(size=6, color='green'),
            name='Synthesized Trajectory'
        ))
        # Mark the goal (blue).
        fig.add_trace(go.Scatter3d(
            x=[goal[0]], y=[goal[1]], z=[goal[2]],
            mode='markers',
            marker=dict(size=10, color='blue'),
            name='Goal'
        ))
        fig.update_layout(
            title=f"Trajectory {t_idx}: Main (Red) and Synthesized (Green)",
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
            margin=dict(l=0, r=0, b=0, t=40)
        )
        fig.write_html(f"synthesized_trajectory_{t_idx}.html")
        fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given cube center (cx,cy,cz) and a rotation (computed with 'zyx' convention),
    compute the eight corner points of the cube.
    
    Returns:
      An (8 x 3) numpy array of candidate viewpoint positions.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker
###############################################################################

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    Assumptions:
      - camera_pose: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x‑axis.
      - RPY values are interpreted using the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Candidate Generation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x,y,z,roll,pitch,yaw] into a 7-tuple camera pose
    (x,y,z,qx,qy,qz,qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

# Map parameter names to indices.
param_to_index = {'x':0, 'y':1, 'z':2, 'roll':3, 'pitch':4, 'yaw':5}

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses
    using the cube-corner method with random variations. Only return candidates that
    satisfy the FoV check.
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    max_pos_range = 0.05      # meters
    max_ang_range = 10        # degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        az_var = az + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        droll = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dyaw = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

###############################################################################
# 5) Trajectory Tree Generation by Connecting Candidate Sets
###############################################################################

def build_trajectories(candidates_by_point, current_index, current_traj, threshold, branch_factor):
    """
    Recursively build all complete trajectories from candidates_by_point starting at current_index.
    Two candidate poses are connected if the Euclidean distance between their positions is below 'threshold'.
    At each step, only the top 'branch_factor' candidates (with smallest distances from the previous pose)
    are used.
    
    Returns a list of complete trajectories (each is a list of candidate 6D poses).
    """
    if current_index == len(candidates_by_point) - 1:
        return [current_traj]
    
    trajectories = []
    current_candidate = current_traj[-1]
    p_pos = np.array(current_candidate[:3])
    next_candidates = candidates_by_point.get(current_index + 1, [])
    
    # Compute distances from current candidate to each candidate in the next set.
    cand_with_distance = []
    for cand in next_candidates:
        cand_pos = np.array(cand[:3])
        dist = np.linalg.norm(cand_pos - p_pos)
        if dist <= threshold:
            cand_with_distance.append((cand, dist))
    # Sort candidates by distance.
    cand_with_distance.sort(key=lambda tup: tup[1])
    
    # Limit branching.
    for cand, _ in cand_with_distance[:branch_factor]:
        new_traj = current_traj + [cand]
        sub_trajs = build_trajectories(candidates_by_point, current_index + 1, new_traj, threshold, branch_factor)
        trajectories.extend(sub_trajs)
    return trajectories

###############################################################################
# 6) Main: Generate Multiple Trajectories and Plot Them Separately
###############################################################################

def main():
    # Load main trajectory data.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Process the first demo.
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = demo_data["obs"]["joint_states"][:]  # shape: (T, 7)
                positions_list.append(robot_states[:, :3])
                orientations_list.append(robot_states[:, 3:6])
    
    # Form main trajectory as list of 6D poses.
    main_traj = []
    for pos, rpy in zip(positions_list[0], orientations_list[0]):
        main_traj.append(list(pos) + list(rpy))
    
    # Define the goal as the last point's position.
    goal = positions_list[0][-1]
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # For each main trajectory point, generate candidate poses.
    candidates_by_point = {}
    for i, base_pose in enumerate(main_traj):
        cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=10)
        candidates_by_point[i] = cands
    
    # Now, build a tree of trajectories by connecting candidates from consecutive points.
    # We use a distance threshold (e.g., 0.01 m) and allow up to 'branch_factor' choices at each step.
    threshold = 0.01
    branch_factor = 3
    all_trajectories = []
    # For each candidate at the first point, build complete trajectories.
    for cand in candidates_by_point.get(0, []):
        trajs = build_trajectories(candidates_by_point, 0, [cand], threshold, branch_factor)
        all_trajectories.extend(trajs)
    
    # As a fallback, if no candidate tree was generated, use the main trajectory.
    if not all_trajectories:
        all_trajectories = [main_traj]
    
    # Ensure each trajectory ends at the goal.
    for traj in all_trajectories:
        traj[-1][:3] = list(goal)
    
    # Plot each complete trajectory in a different color.
    colors = ['green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'brown', 'olive']
    fig = go.Figure()
    for idx, traj in enumerate(all_trajectories):
        traj_x = [p[0] for p in traj]
        traj_y = [p[1] for p in traj]
        traj_z = [p[2] for p in traj]
        fig.add_trace(go.Scatter3d(
            x=traj_x, y=traj_y, z=traj_z,
            mode='lines+markers',
            marker=dict(size=4, color=colors[idx % len(colors)]),
            name=f'Synthesized Traj {idx}'
        ))
    
    # Also plot the main trajectory in a specific color.
    main_x = [p[0] for p in main_traj]
    main_y = [p[1] for p in main_traj]
    main_z = [p[2] for p in main_traj]
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        marker=dict(size=6, color='red'),
        name='Main Trajectory'
    ))
    # Mark the goal.
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=10, color='black'),
        name='Goal'
    ))
    fig.update_layout(
        title="Multiple Synthesized Trajectories (Different Colors) and Main Trajectory (Red)",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("multiple_synthesized_trajectories.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given cube center (cx,cy,cz) and a rotation (using 'zyx' convention),
    compute the eight corner points of the cube.
    
    Returns:
      An (8 x 3) numpy array of candidate viewpoint positions.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker
###############################################################################

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose is a 7-tuple: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x‑axis.
      - RPY values are interpreted using the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Candidate Generation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x,y,z,roll,pitch,yaw] into a 7-tuple camera pose
    (x,y,z,qx,qy,qz,qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

# Map parameter names to indices.
param_to_index = {'x':0, 'y':1, 'z':2, 'roll':3, 'pitch':4, 'yaw':5}

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=20):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses
    using the cube-corner method with random variations. Only return candidates that
    satisfy the FoV check.
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    max_pos_range = 0.05      # meters
    max_ang_range = 10        # degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        az_var = az + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        droll = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dyaw = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

###############################################################################
# 5) Trajectory Tree Generation by Connecting Candidate Sets
###############################################################################

def build_trajectories(candidates_by_point, current_index, current_traj, threshold, branch_factor):
    """
    Recursively build complete trajectories from candidates_by_point starting at current_index.
    Two candidate poses are connected if the Euclidean distance between their positions is below 'threshold'.
    At each step, only the top 'branch_factor' candidates (with smallest distances from the previous pose)
    are used.
    
    Returns a list of complete trajectories (each is a list of candidate 6D poses).
    """
    if current_index == len(candidates_by_point) - 1:
        return [current_traj]
    
    trajectories = []
    current_candidate = current_traj[-1]
    p_pos = np.array(current_candidate[:3])
    next_candidates = candidates_by_point.get(current_index + 1, [])
    
    cand_with_distance = []
    for cand in next_candidates:
        cand_pos = np.array(cand[:3])
        dist = np.linalg.norm(cand_pos - p_pos)
        if dist <= threshold:
            cand_with_distance.append((cand, dist))
    cand_with_distance.sort(key=lambda tup: tup[1])
    
    for cand, _ in cand_with_distance[:branch_factor]:
        new_traj = current_traj + [cand]
        sub_trajs = build_trajectories(candidates_by_point, current_index + 1, new_traj, threshold, branch_factor)
        trajectories.extend(sub_trajs)
    return trajectories

###############################################################################
# 6) Main: Generate Multiple Trajectories and Plot Them Separately
###############################################################################

def main():
    # Load main trajectory data.
    file_path = "demo_duck_feb12.hdf5"
    main_trajs = []  # Each main trajectory is a list of 6D poses.
    goals = []       # The goal (last point) for each trajectory.
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T,7)
                traj = []
                for pos, rpy in zip(robot_states[:, :3], robot_states[:, 3:6]):
                    traj.append(list(pos) + list(rpy))
                main_trajs.append(traj)
                goals.append(robot_states[-1, :3])
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # For each main trajectory, generate candidates for each point.
    all_synthesized = []
    for traj, goal in zip(main_trajs, goals):
        candidates_by_point = {}
        for i, base_pose in enumerate(traj):
            cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=20)
            candidates_by_point[i] = cands
            print(f"Main Traj point {i}: {len(cands)} candidates generated.")
        
        # Build trajectory tree by connecting candidates from consecutive points.
        threshold = 0.02  # Increased threshold to allow connection.
        branch_factor = 3
        traj_candidates = []
        # For each candidate at the first point.
        for cand in candidates_by_point.get(0, []):
            trajs = build_trajectories(candidates_by_point, 0, [cand], threshold, branch_factor)
            traj_candidates.extend(trajs)
        # Fallback: if no candidate tree was generated, use main trajectory.
        if not traj_candidates:
            traj_candidates = [traj]
        
        # Force the last point of each trajectory to be exactly the goal.
        for t in traj_candidates:
            t[-1][:3] = list(goal)
        
        all_synthesized.append(traj_candidates)
    
    # Plot each main trajectory and its synthesized trajectories separately.
    colors = ['green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'brown', 'olive']
    print("Total synthesized trajectory: ",len(all_synthesized))
    for m_idx, (main_traj, traj_candidates, goal) in enumerate(zip(main_trajs, all_synthesized, goals)):
        fig = go.Figure()
        # Plot main trajectory in red.
        main_x = [p[0] for p in main_traj]
        main_y = [p[1] for p in main_traj]
        main_z = [p[2] for p in main_traj]
        fig.add_trace(go.Scatter3d(
            x=main_x, y=main_y, z=main_z,
            mode='lines+markers',
            marker=dict(size=6, color='red'),
            name='Main Trajectory'
        ))
        # Plot each synthesized trajectory (complete chain) in a different color.
        for idx, synth_traj in enumerate(traj_candidates):
            synth_x = [p[0] for p in synth_traj]
            synth_y = [p[1] for p in synth_traj]
            synth_z = [p[2] for p in synth_traj]
            fig.add_trace(go.Scatter3d(
                x=synth_x, y=synth_y, z=synth_z,
                mode='lines+markers',
                marker=dict(size=4, color=colors[idx % len(colors)]),
                name=f'Synthesized Traj {idx}'
            ))
        # Mark the goal.
        fig.add_trace(go.Scatter3d(
            x=[goal[0]], y=[goal[1]], z=[goal[2]],
            mode='markers',
            marker=dict(size=10, color='black'),
            name='Goal'
        ))
        fig.update_layout(
            title=f"Main Trajectory {m_idx} and Synthesized Trajectories",
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
            margin=dict(l=0, r=0, b=0, t=40)
        )
        fig.write_html(f"synthesized_trajectory_main_{m_idx}.html")
        fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Cube Corners as Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given cube center (cx,cy,cz) and a rotation (using 'zyx' convention),
    compute the eight corner points of the cube.
    
    Returns:
      An (8 x 3) numpy array of candidate viewpoint positions.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker
###############################################################################

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose is a 7-tuple: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x‑axis.
      - RPY values are interpreted using the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility Functions for Pose Conversion and Candidate Generation
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x,y,z,roll,pitch,yaw] into a 7-tuple camera pose
    (x,y,z,qx,qy,qz,qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

param_to_index = {'x':0, 'y':1, 'z':2, 'roll':3, 'pitch':4, 'yaw':5}

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=20):
    """
    For a given main trajectory point (base_pose, a 6D pose), generate candidate poses
    using the cube-corner method with random variations. Only returns candidates that
    satisfy the FoV check.
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    max_pos_range = 0.05      # meters
    max_ang_range = 10        # degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        az_var = az + random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        droll = random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        dyaw = random.uniform(-max_ang_range, max_ang_range) * deg_to_rad
        roll_new = roll + droll
        pitch_new = pitch + dpitch
        yaw_new = yaw + dyaw
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

###############################################################################
# 5) Trajectory Tree Generation by Connecting Candidate Sets
###############################################################################

def build_trajectories(candidates_by_point, current_index, current_traj, threshold, branch_factor):
    """
    Recursively build complete trajectories from candidates_by_point starting at current_index.
    Two candidate poses are connected if the Euclidean distance between their positions is below 'threshold'.
    At each step, only the top 'branch_factor' candidates (sorted by distance) are used.
    
    Returns a list of complete trajectories (each is a list of candidate 6D poses).
    """
    if current_index == len(candidates_by_point) - 1:
        return [current_traj]
    
    trajectories = []
    current_candidate = current_traj[-1]
    p_pos = np.array(current_candidate[:3])
    next_candidates = candidates_by_point.get(current_index + 1, [])
    
    cand_with_distance = []
    for cand in next_candidates:
        cand_pos = np.array(cand[:3])
        dist = np.linalg.norm(cand_pos - p_pos)
        if dist <= threshold:
            cand_with_distance.append((cand, dist))
    cand_with_distance.sort(key=lambda tup: tup[1])
    
    for cand, _ in cand_with_distance[:branch_factor]:
        new_traj = current_traj + [cand]
        sub_trajs = build_trajectories(candidates_by_point, current_index + 1, new_traj, threshold, branch_factor)
        trajectories.extend(sub_trajs)
    return trajectories

###############################################################################
# 6) Main: Generate Multiple Synthesized Trajectories and Plot in a Single Figure
###############################################################################

def main():
    # Load main trajectories from file.
    file_path = "demo_duck_feb12.hdf5"
    main_trajs = []  # each main trajectory is a list of 6D poses.
    goals = []       # goal (last point) for each trajectory.
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T,7)
                traj = []
                for pos, rpy in zip(robot_states[:, :3], robot_states[:, 3:6]):
                    traj.append(list(pos) + list(rpy))
                main_trajs.append(traj)
                goals.append(robot_states[-1, :3])
    
    # FoV parameters.
    camera_fov = (90, 90)
    
    # For each main trajectory, generate candidate sets.
    all_synthesized = []  # list of lists of synthesized trajectories (per main trajectory).
    for traj, goal in zip(main_trajs, goals):
        candidates_by_point = {}
        for i, base_pose in enumerate(traj):
            cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=20)
            candidates_by_point[i] = cands
            print(f"Main Trajectory point {i}: {len(cands)} candidates.")
        
        # Build trajectory trees by connecting candidates.
        threshold = 0.03  # distance threshold (m)
        branch_factor = 5
        traj_candidates = []
        for cand in candidates_by_point.get(0, []):
            trajs = build_trajectories(candidates_by_point, 0, [cand], threshold, branch_factor)
            traj_candidates.extend(trajs)
        if not traj_candidates:
            traj_candidates = [traj]
        # Force the last point of each candidate trajectory to equal the goal.
        for t in traj_candidates:
            t[-1][:3] = list(goal)
        all_synthesized.extend(traj_candidates)
    
    print("Total synthesized trajectory: ",len(all_synthesized))
    
    # Now, plot everything in a single figure.
    fig = go.Figure()
    # Plot all main trajectories in red (dashed line).
    for traj in main_trajs:
        main_x = [p[0] for p in traj]
        main_y = [p[1] for p in traj]
        main_z = [p[2] for p in traj]
        fig.add_trace(go.Scatter3d(
            x=main_x, y=main_y, z=main_z,
            mode='lines+markers',
            line=dict(color='red', dash='dot'),
            marker=dict(size=6, color='red'),
            name='Main Trajectory'
        ))
    # Plot all synthesized trajectories in different colors.
    colors = ['green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'brown', 'olive']
    for idx, synth_traj in enumerate(all_synthesized):
        synth_x = [p[0] for p in synth_traj]
        synth_y = [p[1] for p in synth_traj]
        synth_z = [p[2] for p in synth_traj]
        fig.add_trace(go.Scatter3d(
            x=synth_x, y=synth_y, z=synth_z,
            mode='lines+markers',
            marker=dict(size=4, color=colors[idx % len(colors)]),
            name=f'Synthesized Traj {idx}'
        ))
    # Plot goal(s) (blue markers).
    for goal in goals:
        fig.add_trace(go.Scatter3d(
            x=[goal[0]], y=[goal[1]], z=[goal[2]],
            mode='markers',
            marker=dict(size=10, color='black'),
            name='Goal'
        ))
    fig.update_layout(
        title="All Main Trajectories (Red, dashed) and Synthesized Trajectories (Colored)",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("all_trajectories_combined.html")
    fig.show()

if __name__ == "__main__":
    main()


In [ ]:
import h5py
import random
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation: Generate Candidate Viewpoints
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    """
    Given the cube center (cx,cy,cz) and a rotation (using 'zyx' convention),
    compute the eight corner points of the cube.
    
    Returns:
      An (8 x 3) numpy array of candidate viewpoint positions.
    """
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated = rotation.apply(corners)
    return rotated + np.array([cx, cy, cz])

###############################################################################
# 3) Field of View Checker
###############################################################################

def is_inFOV(camera_pose, camera_fov, point_obj):
    """
    Check if the goal (point_obj) is within the camera's field of view.
    
    Assumptions:
      - camera_pose is a 7-tuple: (x, y, z, qx, qy, qz, qw)
      - The camera’s optical axis is along the positive x-axis.
      - RPY values are interpreted using the 'zyx' convention.
    """
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return abs(h_angle) <= h_fov/2 and abs(v_angle) <= v_fov/2

###############################################################################
# 4) Utility: Converting a 6D Pose to a Camera Pose
###############################################################################

def candidate_to_cam_pose(candidate):
    """
    Convert a candidate 6D pose [x,y,z,roll,pitch,yaw] into a 7-tuple camera pose
    (x,y,z,qx,qy,qz,qw) using the 'zyx' convention.
    """
    pos = candidate[:3]
    angles = candidate[3:]
    quat = R.from_euler('zyx', angles).as_quat()
    return (pos[0], pos[1], pos[2], quat[0], quat[1], quat[2], quat[3])

###############################################################################
# 5) Generating Candidates per Main Trajectory Point
###############################################################################

def generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=20):
    """
    For a given main trajectory point (base_pose: 6D), generate candidate poses
    by random variation (cube corners, spherical offsets, etc.).
    Only returns candidates that pass the FoV check.
    
    Returns:
      A list of 6D candidate poses, each is [x,y,z,roll,pitch,yaw].
    """
    candidates = []
    deg_to_rad = np.pi / 180.0
    max_pos_range = 0.05      # 5 cm
    max_ang_range = 10        # 10 degrees
    for _ in range(num_samples):
        x, y, z, roll, pitch, yaw = base_pose
        dx = x - goal[0]
        dy = y - goal[1]
        dz = z - goal[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        
        # Random variation in spherical coords
        az_var = az + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        el_var = el + random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal[0] + dx_var
        y_new = goal[1] + dy_var
        z_new = goal[2] + dz_var
        
        # Random variation in orientation
        droll  = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dpitch = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        dyaw   = random.uniform(-max_ang_range, max_ang_range)*deg_to_rad
        roll_new  = roll + droll
        pitch_new = pitch + dpitch
        yaw_new   = yaw + dyaw
        
        candidate = [x_new, y_new, z_new, roll_new, pitch_new, yaw_new]
        cam_pose = candidate_to_cam_pose(candidate)
        if is_inFOV(cam_pose, camera_fov, goal):
            candidates.append(candidate)
    return candidates

###############################################################################
# 6) Building All Candidate Chains (Trajectory Trees)
###############################################################################

def build_trajectories(candidates_by_point, current_index, current_traj, threshold, branch_factor):
    """
    Recursively build complete trajectories from candidates_by_point, starting at 'current_index'.
    We connect a candidate from step i to one from step i+1 if their positions are within 'threshold'.
    We only expand up to 'branch_factor' nearest connections per step.
    
    Returns:
      A list of complete synthesized trajectories (each is a list of 6D poses).
    """
    if current_index == len(candidates_by_point) - 1:
        return [current_traj]
    
    trajectories = []
    prev_cand = current_traj[-1]
    p_pos = np.array(prev_cand[:3])
    next_candidates = candidates_by_point.get(current_index + 1, [])
    
    # For each candidate in the next step, compute distance from prev_cand
    cand_with_dist = []
    for cand in next_candidates:
        c_pos = np.array(cand[:3])
        dist = np.linalg.norm(c_pos - p_pos)
        if dist <= threshold:
            cand_with_dist.append((cand, dist))
    # Sort by distance
    cand_with_dist.sort(key=lambda x: x[1])
    
    # Expand up to 'branch_factor' connections
    for cand, _ in cand_with_dist[:branch_factor]:
        new_traj = current_traj + [cand]
        sub_trajs = build_trajectories(candidates_by_point, current_index+1, new_traj, threshold, branch_factor)
        trajectories.extend(sub_trajs)
    return trajectories

###############################################################################
# 7) MAIN: Produce Multiple Synthesized Trajectories per Main Trajectory, Single Figure
###############################################################################

def main():
    file_path = "demo_duck_feb12.hdf5"
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        
        # Each 'main_traj' is a list of 6D [x,y,z,roll,pitch,yaw].
        all_main_trajs = []
        all_goals = []
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                robot_states = obs_group["joint_states"][:]  # shape: (T,7)
                new_traj = []
                for pos, rpy in zip(robot_states[:, :3], robot_states[:, 3:6]):
                    new_traj.append(list(pos) + list(rpy))
                all_main_trajs.append(new_traj)
                all_goals.append(robot_states[-1, :3])  # last point is the goal
    
    camera_fov = (90, 90)
    threshold  = 0.03   # distance threshold to connect candidates
    branch_factor = 5   # how many expansions to allow
    # We store all main trajectories and their multiple synthesized ones
    # in a single figure.
    fig = go.Figure()
    
    # We'll cycle through multiple colors for the synthesized trajectories
    # We can store them in a single combined list, but let's do them in order for clarity.
    color_list = ['blue','green','orange','purple','magenta','cyan','lime','brown','gray','olive']
    color_idx = 0
    
    # Plot each main trajectory in red (dashed line).
    for main_traj_idx, (main_traj, goal) in enumerate(zip(all_main_trajs, all_goals)):
        # Plot the main trajectory in red, dashed line.
        x_main = [p[0] for p in main_traj]
        y_main = [p[1] for p in main_traj]
        z_main = [p[2] for p in main_traj]
        fig.add_trace(go.Scatter3d(
            x=x_main, y=y_main, z=z_main,
            mode='lines+markers',
            line=dict(color='red', dash='dot'),
            marker=dict(size=6, color='red'),
            name=f"Main Trajectory {main_traj_idx}"
        ))
        
        # Generate candidates for each point.
        candidates_by_point = {}
        for i, base_pose in enumerate(main_traj):
            cands = generate_candidates_for_point(base_pose, goal, camera_fov, num_samples=20)
            candidates_by_point[i] = cands
            print(f"Main Traj {main_traj_idx}, Point {i}: {len(cands)} candidates.")
        
        # Build multiple trajectory chains from these candidates.
        full_trajs = []
        start_candidates = candidates_by_point.get(0, [])
        for cand in start_candidates:
            partial_chains = build_trajectories(candidates_by_point, 0, [cand], threshold, branch_factor)
            full_trajs.extend(partial_chains)
        
        # If no chain was built, just use the main trajectory itself as fallback
        if not full_trajs:
            full_trajs = [main_traj]
        
        # Force each chain's last point to be exactly the goal
        for t in full_trajs:
            t[-1][:3] = goal

        print(len(full_trajs))
        
        # Now plot each chain in a different color, in the same figure
        for chain_idx, chain in enumerate(full_trajs):
            x_syn = [p[0] for p in chain]
            y_syn = [p[1] for p in chain]
            z_syn = [p[2] for p in chain]
            c_col = color_list[color_idx % len(color_list)]
            fig.add_trace(go.Scatter3d(
                x=x_syn, y=y_syn, z=z_syn,
                mode='lines+markers',
                marker=dict(size=4, color=c_col),
                name=f"Synth. Traj. {main_traj_idx}_{chain_idx}"
            ))
            color_idx += 1
        
        # Mark the goal with black
        fig.add_trace(go.Scatter3d(
            x=[goal[0]], y=[goal[1]], z=[goal[2]],
            mode='markers',
            marker=dict(size=10, color='black'),
            name=f"Goal {main_traj_idx}"
        ))
    
    fig.update_layout(
        title="All Main Trajectories (Red, dashed) + Multiple Synthesized (Various Colors)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z"
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.write_html("all_trajectories_with_multiple_synth.html")
    fig.show()

if __name__ == "__main__":
    main()
